In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *

from datetime import datetime, timedelta
import warnings

from minio_extractor import fetch_logs, RequestConfig

# Disable warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

#For widescreen view
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>")) 

spark = SparkSession \
    .builder \
    .appName("Spark_EID1") \
    .config("spark.sql.caseSensitive","True") \
    .config('spark.sql.execution.arrow.pyspark.enabled', True) \
    .config('spark.sql.session.timeZone', 'EST') \
    .config('spark.driver.memory','16G') \
    .config('spark.ui.showConsoleProgress', True) \
    .config('spark.sql.repl.eagerEval.enabled', True) \
    .getOrCreate()
sc = spark.sparkContext
sc.setLogLevel("ERROR")

### reading from a parquet object
park = spark.read.parquet("edr_all_raw.parquet")

In [ ]:
park.printSchema()
park.show(n = 1, vertical = True)
park.createOrReplaceTempView('sparktable')

In [ ]:
sparkDF_EID1 = spark.sql(
'''
SELECT computer_name, ParentImage, ParentCommandLine, Image, CommandLine, ParentProcessGuid, ProcessGuid, OriginalFileName, CurrentDirectory, User
FROM sparktable
WHERE (source_name = "Microsoft-Windows-Sysmon" AND event_id = 1)
'''
)
sparkDF_EID1.show(10,30)

### proc_creation_win_7zip_cve_2022_29072
##### description : 7-Zip through 21.07 on Windows allows privilege escalation (CVE-2022-29072) and command execution when a file with the .7z extension is dragged to the Help>Contents area. This is caused by misconfiguration of 7z.dll and a heap overflow. The command runs in a child process under the 7zFM.exe process.

In [ ]:
huntCase = spark.sql(
'''
SELECT  computer_name, ParentImage, ParentCommandLine, Image, CommandLine, ParentProcessGuid, ProcessGuid, OriginalFileName, CurrentDirectory, User
FROM sparktable
WHERE (source_name = "Microsoft-Windows-Sysmon" AND event_id = 1
    AND Image LIKE '%\\cmd.exe'
    AND ParentImage LIKE '%\\7zFM.exe'
    )
'''
)
huntCase.show(10,40)

### Abused Debug Privilege by Arbitrary Parent Processes
##### description : Detection of unusual child processes by different system processes

In [ ]:
huntCase = spark.sql(
'''
SELECT computer_name, ParentImage, ParentCommandLine, Image, CommandLine, ParentProcessGuid, ProcessGuid, OriginalFileName, CurrentDirectory, User
FROM sparktable
WHERE (source_name = "Microsoft-Windows-Sysmon" AND event_id = 1
    AND ( lower(Image) LIKE '%\\powershell.exe'
        OR lower(Image) LIKE '%\\pwsh.exe'
        OR lower(Image) LIKE '%\\cmd.exe'
    )
    AND (lower(ParentImage) LIKE '%\\winlogon.exe' 
        OR lower(ParentImage) LIKE '%\\services.exe' 
        OR lower(ParentImage) LIKE '%\\lsass.exe' 
        OR lower(ParentImage) LIKE '%\\csrss.exe' 
        OR lower(ParentImage) LIKE '%\\smss.exe' 
        OR lower(ParentImage) LIKE '%\\wininit.exe' 
        OR lower(ParentImage) LIKE '%\\spoolsv.exe' 
        OR lower(ParentImage) LIKE '%\\searchindexer.exe')
    )
    AND (lower(User) LIKE '%authori%'
        OR lower(User) LIKE '%autori%'
    )
    
'''
)
huntCase.show(10,50)

### Abusing Windows Telemetry For Persistence
##### description : Windows telemetry makes use of the binary CompatTelRunner.exe to run a variety of commands and perform the actual telemetry collections. This binary was created to be easily extensible, and to that end, it relies on the registry to instruct on which commands to run. The problem is, it will run any arbitrary command without restriction of location or type.

In [ ]:
huntCase = spark.sql(
'''
SELECT computer_name, ParentImage, ParentCommandLine, Image, CommandLine, OriginalFileName, CurrentDirectory, User
FROM sparktable
WHERE (source_name = "Microsoft-Windows-Sysmon" AND event_id = 1
    AND 
        (
        lower(CommandLine) LIKE '%schtasks%'
        OR lower(CommandLine) LIKE '%\\Application Experience\\Microsoft Compatibility Appraiser%'
        )
    )
    AND NOT 
        (
        lower(User) LIKE '%authori%'
        OR lower(User) LIKE '%autori%'
        )
'''
)
huntCase.show(10,80)

### Accesschk Usage To Check Privileges
##### description : Accesschk is an access and privilege audit tool developed by SysInternal and often being used by attacker to verify privileges

In [ ]:
huntCase = spark.sql(
'''
SELECT computer_name, ParentImage, ParentCommandLine, Image, CommandLine, OriginalFileName, CurrentDirectory, User
FROM sparktable
WHERE (source_name = "Microsoft-Windows-Sysmon" AND event_id = 1
    AND 
        (
        lower(OriginalFileName) LIKE '%AccessChk%'
        OR lower(CommandLine) LIKE '%\\accesschk.exe%'
        )
    AND 
        (
        lower(CommandLine) LIKE '%uwcqv%'
        OR lower(CommandLine) LIKE '%kwsu%'
        OR lower(CommandLine) LIKE '%qwsu%'
        OR lower(CommandLine) LIKE '%uwdqs%'
        )
    )
'''
)
huntCase.show(10,80)

### AdFind Usage Detection
##### description :  AdFind continues to be seen across majority of breaches. It is used to domain trust discovery to plan out subsequent steps in the attack chain.

In [ ]:
huntCase = spark.sql(
'''
SELECT computer_name, ParentImage, ParentCommandLine, Image, CommandLine, OriginalFileName, CurrentDirectory, User
FROM sparktable
WHERE (source_name = "Microsoft-Windows-Sysmon" AND event_id = 1
    AND 
        (
        lower(CommandLine) LIKE '%domainlist%'
        OR lower(CommandLine) LIKE '%trustdmp%'
        OR lower(CommandLine) LIKE '%dcmodes%'
        OR lower(CommandLine) LIKE '%adinfo%'
        OR lower(CommandLine) LIKE '% dclist %'
        OR lower(CommandLine) LIKE '%computer_pwdnotreqd%'
        OR lower(CommandLine) LIKE '%objectcategory=%'
        OR lower(CommandLine) LIKE '%-subnets -f%'
        OR lower(CommandLine) LIKE '%name=\"Domain Admins\"%'
        OR lower(CommandLine) LIKE '%-sc u:%'
        OR lower(CommandLine) LIKE '%domainncs%'
        OR lower(CommandLine) LIKE '%dompol%'
        OR lower(CommandLine) LIKE '% oudmp %'
        OR lower(CommandLine) LIKE '%subnetdmp%'
        OR lower(CommandLine) LIKE '%gpodmp%'
        OR lower(CommandLine) LIKE '%fspdmp%'
        OR lower(CommandLine) LIKE '%users_noexpire%'
        OR lower(CommandLine) LIKE '%computers_active%'
        )
    )
'''
)
huntCase.show(10,50)

### Advanced IP Scanner
##### description :  Detects the use of Advanced IP Scanner. Seems to be a popular tool for ransomware groups.

In [ ]:
huntCase = spark.sql(
'''
SELECT computer_name, ParentImage, ParentCommandLine, Image, CommandLine, OriginalFileName, CurrentDirectory, User
FROM sparktable
WHERE (source_name = "Microsoft-Windows-Sysmon" AND event_id = 1
    AND 
        (
            lower(Image) LIKE '%\\advanced_ip_scanner%'
            OR lower(Image) LIKE '%\\advanced_port_scanner%'
        )
    AND 
        (
            lower(CommandLine) LIKE '%portable%'
            OR lower(CommandLine) LIKE '%lng%'
        )
    )
'''
)
huntCase.show(10,70)

### Execute From Alternate Data Streams
##### description :  Adversaries may use NTFS file attributes to hide their malicious data in order to evade detection

In [ ]:
huntCase = spark.sql(
'''
SELECT computer_name, ParentImage, ParentCommandLine, Image, CommandLine, OriginalFileName, CurrentDirectory, User
FROM sparktable
WHERE (source_name = "Microsoft-Windows-Sysmon" AND event_id = 1
    AND
        (
            lower(CommandLine) LIKE '%txt:%'
        )
    AND
        (
            (
                lower(CommandLine) LIKE '%type %'
                AND  lower(CommandLine) LIKE '% > %'
            )
        OR
            (
                lower(CommandLine) LIKE '%makecab %'
                AND  lower(CommandLine) LIKE '%.cab%'
            )
        OR
            (
                lower(CommandLine) LIKE '%reg %'
                AND  lower(CommandLine) LIKE '% export %'
            )
        OR
            (
                lower(CommandLine) LIKE '%regedit %'
                AND  lower(CommandLine) LIKE '% /E %'
            )
        OR
            (
                lower(CommandLine) LIKE '%esentutl %'
                AND  lower(CommandLine) LIKE '% /y %'
                AND  lower(CommandLine) LIKE '% /d %'
                AND  lower(CommandLine) LIKE '% /o %'
            )
        )

    )
'''
)
huntCase.show(10,70)

### Always Install Elevated MSI Spawned Cmd And Powershell
##### description :  This rule looks for Windows Installer service (msiexec.exe) spawned command line and/or powershell

In [ ]:
huntCase = spark.sql(
'''
SELECT computer_name, ParentImage, ParentCommandLine, Image, CommandLine, ParentProcessGuid, ProcessGuid, OriginalFileName, CurrentDirectory, User
FROM sparktable
WHERE (source_name = "Microsoft-Windows-Sysmon" AND event_id = 1
    AND 
        ( 
            lower(Image) LIKE '%\\powershell.exe'
            OR lower(Image) LIKE '%\\pwsh.exe'
            OR lower(Image) LIKE '%\\cmd.exe'
        )
    AND
        (
            (
                lower(ParentImage) LIKE '%\\Windows\\Installer\\%'
                AND  lower(ParentImage) LIKE '%msi%'
            )
        AND
            (
                lower(ParentImage) LIKE '%tmp'
            )
        )

    )
'''
)
huntCase.show(10,70)

### Always Install Elevated Windows Installer
##### description :  This rule looks for Windows Installer service (msiexec.exe) trying to install MSI packages with SYSTEM privilege

In [ ]:
huntCase = spark.sql(
'''
SELECT computer_name, ParentImage, ParentCommandLine, Image, ParentProcessGuid, ProcessGuid, CommandLine, OriginalFileName, CurrentDirectory, User
FROM sparktable
WHERE (source_name = "Microsoft-Windows-Sysmon" AND event_id = 1
    AND
        (
            lower(User) LIKE '%authori%'
            OR lower(User) LIKE '%autori%'
        )
    AND (
            (
                (
                    lower(Image) LIKE '%\\Windows\\Installer\\%'
                    AND  lower(Image) LIKE '%msi%'
                )
            AND
                (
                    lower(ParentImage) LIKE '%tmp'
                )
            AND NOT
                    (
                        lower(ParentImage) LIKE  '%c:%windows%services.exe%'
                    )
            )
        OR
            (
                    lower(Image) LIKE '%\\msiexec.exe%'
                    AND NOT lower(ParentImage) LIKE '%msi%'

            AND
                    (
                        lower(User) LIKE '%authori%'
                        OR lower(User) LIKE '%autori%'
                    )
            AND NOT
                    (
                        lower(ParentImage) LIKE  '%c:%windows%services.exe%'
                    )
            )
        )
    )
'''
)
huntCase.show(10,70)

### Scheduled Task WScript VBScript
##### description :  Detects specific process parameters as used by ACTINIUM scheduled task persistence creation.

In [ ]:
huntCase = spark.sql(
'''
SELECT computer_name, ParentImage, ParentCommandLine, Image, CommandLine, ParentProcessGuid, ProcessGuid, OriginalFileName, CurrentDirectory, User
FROM sparktable
WHERE (source_name = "Microsoft-Windows-Sysmon" AND event_id = 1
    AND
        (
            lower(CommandLine) LIKE '%\\schtasks%'
            AND lower(CommandLine) LIKE '%e:vbscript%'
            AND lower(CommandLine) LIKE '%wscript%'
            AND lower(CommandLine) LIKE '%create%'
        )
    )
'''
)
huntCase.show(10,70)

### APT29
##### description :  This method detects a suspicious PowerShell command line combination as used by APT29 in a campaign against U.S. think tanks.

In [ ]:
huntCase = spark.sql(
'''
SELECT computer_name, ParentImage, ParentCommandLine, Image, CommandLine, ParentProcessGuid, ProcessGuid, OriginalFileName, CurrentDirectory, User
FROM sparktable
WHERE (source_name = "Microsoft-Windows-Sysmon" AND event_id = 1
    AND
        (
            lower(CommandLine) LIKE '%-noni%'
            AND lower(CommandLine) LIKE '%-ep%'
            AND lower(CommandLine) LIKE '%bypass%'
            AND lower(CommandLine) LIKE '%$%'
        )
    )
'''
)
huntCase.show(10,70)

### Baby Shark Activity
##### description :  Detects activity that could be related to Baby Shark malware

In [ ]:
huntCase = spark.sql(
'''
SELECT computer_name, ParentImage, ParentCommandLine, Image, CommandLine, ParentProcessGuid, ProcessGuid, OriginalFileName, CurrentDirectory, User
FROM sparktable
WHERE (source_name = "Microsoft-Windows-Sysmon" AND event_id = 1
        AND
        (
            lower(Image) LIKE '%reg%Software%Terminal%Server%Client%Default%'
            OR lower(CommandLine) LIKE 'powershell.exe%mshta.exe%http%'
            OR lower(CommandLine) LIKE 'psh%mshta.exe%http%'
            OR lower(CommandLine) LIKE '%cmd.exe%taskkill%/im%cmd.exe%'
        )

    )
'''
)
huntCase.show(10,70)

### Judgement Panda Credential Access Activity
##### description : Detects Russian group activity as described in Global Threat Report 2019 by Crowdstrike

In [ ]:
huntCase = spark.sql(
'''
SELECT computer_name, ParentImage, ParentCommandLine, Image, CommandLine, ParentProcessGuid, ProcessGuid, OriginalFileName, CurrentDirectory, User
FROM sparktable
WHERE (source_name = "Microsoft-Windows-Sysmon" AND event_id = 1
        AND
        (
            lower(Image) LIKE '%\\xcopy.exe'
            AND lower(CommandLine) LIKE '%/S%'
            AND lower(CommandLine) LIKE '%/E%'
            AND lower(CommandLine) LIKE '%/C%'
            AND lower(CommandLine) LIKE '%/Q%'
            AND lower(CommandLine) LIKE '%/H%'
            AND lower(CommandLine) LIKE '%\\\\%'
        )
        OR
        (
            lower(Image) LIKE '%\\adexplorer.exe'
            AND lower(CommandLine) LIKE '%-snapshot%'
            AND lower(CommandLine) LIKE '%""%'
            AND lower(CommandLine) LIKE '%c:\\users\\%'
        )

    )
'''
)
huntCase.show(10,70)

### Arbitrary Shell Command Execution Via Settingcontent-Ms
##### description :  The .SettingContent-ms file type was introduced in Windows 10 and allows a user to create "shortcuts" to various Windows 10 setting pages. These files are simply XML and contain paths to various Windows 10 settings binaries.

In [ ]:
huntCase = spark.sql(
'''
SELECT computer_name, ParentImage, ParentCommandLine, Image, CommandLine, ParentProcessGuid, ProcessGuid, OriginalFileName, CurrentDirectory, User
FROM sparktable
WHERE (source_name = "Microsoft-Windows-Sysmon" AND event_id = 1
        AND
        (
            lower(ParentImage) LIKE '%\\Winrar.exe%'
            OR lower(ParentImage) LIKE '%\\7zFM.exe%'
            OR lower(ParentImage) LIKE '%\\peazip.exe%'
        )
        AND
        (
            lower(Image) LIKE '%\\isoburn.exe%'
            OR lower(Image) LIKE '%\\PowerISO.exe%'
            OR lower(Image) LIKE '%\\ImgBurn.exe%'
        )
    )
'''
)
huntCase.show(10,70)

### Using AppVLP To Circumvent ASR File Path Rule
##### description : Application Virtualization Utility is included with Microsoft Office. We are able to abuse "AppVLP" to execute shell commands.
######    Normally, this binary is used for Application Virtualization, but we can use it as an abuse binary to circumvent the ASR file path rule folder or to mark a file as a system file.

In [ ]:
huntCase = spark.sql(
'''
SELECT computer_name, ParentImage, ParentCommandLine, Image, CommandLine, OriginalFileName, CurrentDirectory, User
FROM sparktable
WHERE (source_name = "Microsoft-Windows-Sysmon" AND event_id = 1
        AND lower(CommandLine) LIKE '%appvlp.exe%'
        AND
        (
            lower(CommandLine) LIKE '%cmd.exe%'
            OR lower(CommandLine) LIKE '%powershell.exe%'
            OR lower(CommandLine) LIKE '%pwsh.exe%'
        )
        AND
        (
            lower(CommandLine) LIKE '%.sh%'
            OR lower(CommandLine) LIKE '%.exe%'
            OR lower(CommandLine) LIKE '%.dll%'
            OR lower(CommandLine) LIKE '%.bin%'
            OR lower(CommandLine) LIKE '%.bat%'
            OR lower(CommandLine) LIKE '%.cmd%'
            OR lower(CommandLine) LIKE '%.js%'
            OR lower(CommandLine) LIKE '%.msh%'
            OR lower(CommandLine) LIKE '%.reg%'
            OR lower(CommandLine) LIKE '%.scr%'
            OR lower(CommandLine) LIKE '%.ps%'
            OR lower(CommandLine) LIKE '%.vb%'
            OR lower(CommandLine) LIKE '%.jar%'
            OR lower(CommandLine) LIKE '%.pl%'
            OR lower(CommandLine) LIKE '%.inf%'
            OR lower(CommandLine) LIKE '%.cpl%'
            OR lower(CommandLine) LIKE '%.xml%'
        )
    )
'''
)
huntCase.show(10, 50)

### Atlassian Confluence CVE-2021-26084
##### description : Detects spawning of suspicious child processes by Atlassian Confluence server which may indicate successful exploitation of CVE-2021-26084

In [ ]:
huntCase = spark.sql(
'''
SELECT computer_name, ParentImage, ParentCommandLine, Image, CommandLine, OriginalFileName, CurrentDirectory, User
FROM sparktable
WHERE (source_name = "Microsoft-Windows-Sysmon" AND event_id = 1
        AND
            lower(ParentImage) LIKE '%\\Atlassian%\\java.exe%'
        AND
        (
            lower(CommandLine) LIKE '%.sh%'
            OR lower(CommandLine) LIKE '%cmd /c%'
            OR lower(CommandLine) LIKE '%cmd /k%'
            OR lower(CommandLine) LIKE '%psh%'
            OR lower(CommandLine) LIKE '%powershell%'
            OR lower(CommandLine) LIKE '%certutil%'
            OR lower(CommandLine) LIKE '%rundll32%'
            OR lower(CommandLine) LIKE '%curl%'
            OR lower(CommandLine) LIKE '%ipconfig%'
            OR lower(CommandLine) LIKE '%ping%'
            OR lower(CommandLine) LIKE '%whoami%'
        )
    )
'''
)
huntCase.show(10, 50)

### Hiding Files with Attrib.exe
##### description : Detects usage of attrib.exe to hide files from users.

In [ ]:
huntCase = spark.sql(
'''
SELECT computer_name, ParentImage, ParentCommandLine, Image, CommandLine, OriginalFileName, CurrentDirectory, User
FROM sparktable
WHERE (source_name = "Microsoft-Windows-Sysmon" AND event_id = 1
        AND
            lower(Image) LIKE '%\attrib.exe%'
        AND
            lower(CommandLine) LIKE '% +h %' 
            AND NOT (
                        lower(CommandLine) LIKE '%\\desktop.ini %'
                        OR lower(ParentImage) LIKE '%cmd.exe%'
                        OR lower(CommandLine) LIKE '%+R%+H%+S%+A%.cui%'
                        OR lower(ParentCommandLine) LIKE '%C%WINDOWS%system32%.bat%'
                     )
    )
'''
)
huntCase.show(10, 50)

### Automated Collection Command Prompt
##### description : Once established within a system or network, an adversary may use automated techniques for collecting internal data.

In [ ]:
huntCase = spark.sql(
'''
SELECT computer_name, ParentImage, ParentCommandLine, Image, CommandLine, OriginalFileName, CurrentDirectory, User
FROM sparktable
WHERE (source_name = "Microsoft-Windows-Sysmon" AND event_id = 1
        AND
        (
            lower(CommandLine) LIKE '%doc%'
            OR lower(CommandLine) LIKE '%docx%'
            OR lower(CommandLine) LIKE '%xls%'
            OR lower(CommandLine) LIKE '%xlsx%'
            OR lower(CommandLine) LIKE '%ppt%'
            OR lower(CommandLine) LIKE '%pptx%'
            OR lower(CommandLine) LIKE '%rtf%'
            OR lower(CommandLine) LIKE '%pdf%'
            OR lower(CommandLine) LIKE '%txt%'
        )
        AND
        (
            (
                lower(CommandLine) LIKE '%dir%'
                AND lower(CommandLine) LIKE '%/b%'
                AND lower(CommandLine) LIKE '%/s%'
            )
            OR 
                lower(OriginalFileName) LIKE '%FINDSTR%'
        )
    )
'''
)
huntCase.show(10, 75)

### Malicious Base64 Encoded Powershell Invoke Cmdlets
##### description : A. Detects base64 encoded powershell cmdlet invocation of known suspicious cmdlets
##### B. Detects base64 encoded listing Win32_Shadowcopy

In [ ]:
huntCase = spark.sql(
'''
SELECT computer_name, ParentImage, ParentCommandLine, Image, CommandLine, OriginalFileName, CurrentDirectory, User
FROM sparktable
WHERE (source_name = "Microsoft-Windows-Sysmon" AND event_id = 1
        AND
        (
            lower(CommandLine) LIKE '%SQBuAHYAbwBrAGUALQBCAGwAbwBvAGQASABvAHUAbgBkA%'
            OR lower(CommandLine) LIKE '%kAbgB2AG8AawBlAC0AQgBsAG8AbwBkAEgAbwB1AG4AZA%'
            OR lower(CommandLine) LIKE '%JAG4AdgBvAGsAZQAtAEIAbABvAG8AZABIAG8AdQBuAGQA%'
            OR lower(CommandLine) LIKE '%SQBuAHYAbwBrAGUALQBNAGkAbQBpAGsAYQB0AHoA%'
            OR lower(CommandLine) LIKE '%kAbgB2AG8AawBlAC0ATQBpAG0AaQBrAGEAdAB6A%'
            OR lower(CommandLine) LIKE '%JAG4AdgBvAGsAZQAtAE0AaQBtAGkAawBhAHQAeg%'
            OR lower(CommandLine) LIKE '%SQBuAHYAbwBrAGUALQBXAE0ASQBFAHgAZQBjA%'
            OR lower(CommandLine) LIKE '%kAbgB2AG8AawBlAC0AVwBNAEkARQB4AGUAYw%'
            OR lower(CommandLine) LIKE '%JAG4AdgBvAGsAZQAtAFcATQBJAEUAeABlAGMA%'
            OR lower(CommandLine) LIKE '%VwBpAG4AMwAyAF8AUwBoAGEAZABvAHcAYwBvAHAAeQAgAHwAIABGAG8AcgBFAGEAYwBoAC0ATwBiAGoAZQBjAHQA%'
            OR lower(CommandLine) LIKE '%cAaQBuADMAMgBfAFMAaABhAGQAbwB3AGMAbwBwAHkAIAB8ACAARgBvAHIARQBhAGMAaAAtAE8AYgBqAGUAYwB0A%'
            OR lower(CommandLine) LIKE '%XAGkAbgAzADIAXwBTAGgAYQBkAG8AdwBjAG8AcAB5ACAAfAAgAEYAbwByAEUAYQBjAGgALQBPAGIAagBlAGMAdA%'
            OR lower(CommandLine) LIKE '%WwBSAGUAZgBsAGUAYwB0AGkAbwBuAC4AQQBzAHMAZQBtAGIAbAB5AF0AOgA6AEwAbwBhAGQAKA%'
            OR lower(CommandLine) LIKE '%sAUgBlAGYAbABlAGMAdABpAG8AbgAuAEEAcwBzAGUAbQBiAGwAeQBdADoAOgBMAG8AYQBkACgA%'
            OR lower(CommandLine) LIKE '%bAFIAZQBmAGwAZQBjAHQAaQBvAG4ALgBBAHMAcwBlAG0AYgBsAHkAXQA6ADoATABvAGEAZAAoA%'
            OR lower(CommandLine) LIKE '%AFsAcgBlAGYAbABlAGMAdABpAG8AbgAuAGEAcwBzAGUAbQBiAGwAeQBdADoAOgAoACIATABvAGEAZAAiAC%'
            OR lower(CommandLine) LIKE '%BbAHIAZQBmAGwAZQBjAHQAaQBvAG4ALgBhAHMAcwBlAG0AYgBsAHkAXQA6ADoAKAAiAEwAbwBhAGQAIgAp%'
            OR lower(CommandLine) LIKE '%AWwByAGUAZgBsAGUAYwB0AGkAbwBuAC4AYQBzAHMAZQBtAGIAbAB5AF0AOgA6ACgAIgBMAG8AYQBkACIAK%'
            OR lower(CommandLine) LIKE '%sAUgBlAGYAbABlAGMAdABpAG8AbgAuAEEAcwBzAGUAbQBiAGwAeQBdADoAOgAoACIATABvAGEAZAAiACkA%'
            OR lower(CommandLine) LIKE '%bAFIAZQBmAGwAZQBjAHQAaQBvAG4ALgBBAHMAcwBlAG0AYgBsAHkAXQA6ADoAKAAiAEwAbwBhAGQAIgApA%'
            OR lower(CommandLine) LIKE '%WwBSAGUAZgBsAGUAYwB0AGkAbwBuAC4AQQBzAHMAZQBtAGIAbAB5AF0AOgA6ACgAIgBMAG8AYQBkACIAKQ%'
            OR lower(CommandLine) LIKE '%WwByAGUAZgBsAGUAYwB0AGkAbwBuAC4AYQBzAHMAZQBtAGIAbAB5AF0AOgA6AEwAbwBhAGQAKA%'
            OR lower(CommandLine) LIKE '%sAcgBlAGYAbABlAGMAdABpAG8AbgAuAGEAcwBzAGUAbQBiAGwAeQBdADoAOgBMAG8AYQBkACgA%'
            OR lower(CommandLine) LIKE '%bAHIAZQBmAGwAZQBjAHQAaQBvAG4ALgBhAHMAcwBlAG0AYgBsAHkAXQA6ADoATABvAGEAZAAoA%'
        )

    )
'''
)
huntCase.show(10, 75)

### Bitsadmin Suspicious Usgae
##### description : Detects suspicious usage of bitsadmin

In [ ]:
huntCase = spark.sql(
'''
SELECT computer_name, ParentImage, ParentCommandLine, Image, CommandLine, OriginalFileName, CurrentDirectory, User
FROM sparktable
WHERE (source_name = "Microsoft-Windows-Sysmon" AND event_id = 1
        AND
        (
            lower(Image) LIKE '%bitsadmin.exe%'
            AND
            (
                lower(CommandLine) LIKE '%http%'
                AND
                (
                    lower(CommandLine) LIKE '%/create %'
                    OR lower(CommandLine) LIKE '%/addfile %'
                )
                OR
                (
                    lower(CommandLine) LIKE '%transfer %'
                )
            )
        )
        OR
            (
                lower(CommandLine) LIKE '%bitsadmin.exe%'
                AND
                    (
                        lower(CommandLine) LIKE '%copy%'
                        OR lower(CommandLine) LIKE '%transfer%'
                        OR lower(CommandLine) LIKE '%create%'
                        OR lower(CommandLine) LIKE '%/addfile %'
                        AND
                            (
                                lower(CommandLine) LIKE '%.asax%'
                                OR lower(CommandLine) LIKE '%.ashx%'
                                OR lower(CommandLine) LIKE '%.asmx%'
                                OR lower(CommandLine) LIKE '%.asp%'
                                OR lower(CommandLine) LIKE '%.aspx%'
                                OR lower(CommandLine) LIKE '%.bat%'
                                OR lower(CommandLine) LIKE '%.cfm%'
                                OR lower(CommandLine) LIKE '%.cgi%'
                                OR lower(CommandLine) LIKE '%.chm%'
                                OR lower(CommandLine) LIKE '%.cmd%'
                                OR lower(CommandLine) LIKE '%.cpl%'
                                OR lower(CommandLine) LIKE '%.gif%'
                                OR lower(CommandLine) LIKE '%.jpeg%'
                                OR lower(CommandLine) LIKE '%.jpg%'
                                OR lower(CommandLine) LIKE '%.jsp%'
                                OR lower(CommandLine) LIKE '%.jspx%'
                                OR lower(CommandLine) LIKE '%.png%'
                                OR lower(CommandLine) LIKE '%.ps1%'
                                OR lower(CommandLine) LIKE '%.psm1%'
                                OR lower(CommandLine) LIKE '%.scf%'
                                OR lower(CommandLine) LIKE '%.sct%'
                                OR lower(CommandLine) LIKE '%.txt%'
                                OR lower(CommandLine) LIKE '%.vbe%'
                                OR lower(CommandLine) LIKE '%.vbs%'
                                OR lower(CommandLine) LIKE '%.war%'
                                OR lower(CommandLine) LIKE '%.wsf%'
                                OR lower(CommandLine) LIKE '%.wsh%'
                                OR lower(CommandLine) LIKE '%.zip%'
                                OR lower(CommandLine) LIKE '%.rar%'
                                OR lower(CommandLine) LIKE '%.dll%'
                            )
                    )
            )

    )
'''
)
huntCase.show(10, 80)

### Modification of Boot Configuration
##### description : Identifies use of the bcdedit command to delete boot configuration data. This tactic is sometimes used as by malware or an attacker as a destructive technique.

In [ ]:
huntCase = spark.sql(
'''
SELECT computer_name, ParentImage, ParentCommandLine, Image, CommandLine, OriginalFileName, CurrentDirectory, User
FROM sparktable
WHERE (source_name = "Microsoft-Windows-Sysmon" AND event_id = 1
        AND
        (
            lower(Image) LIKE '%\\bcdedit.exe%'
            AND lower(CommandLine) LIKE '%set%'
        )
        AND
        (
            (
                lower(CommandLine) LIKE '%bootstatuspolicy%'
                AND lower(CommandLine) LIKE '%ignoreallfailures%'
            )
            OR
            (
                lower(CommandLine) LIKE '%recoveryenabled%'
                AND lower(CommandLine) LIKE '%no%'
            )
        )
    )
'''
)
huntCase.show(10, 75)

### Suspicious Load DLL via CertOC.exe
##### description : Detects when a user installs certificates by using CertOC.exe to loads the target DLL file.

In [ ]:
huntCase = spark.sql(
'''
SELECT computer_name, ParentImage, ParentCommandLine, Image, CommandLine, OriginalFileName, CurrentDirectory, User
FROM sparktable
WHERE (source_name = "Microsoft-Windows-Sysmon" AND event_id = 1
        AND
        (
            lower(Image) LIKE '%\\certoc.exe%'
            OR lower(OriginalFileName) LIKE '%CertOC.exe%'
        )
        AND
        (
            lower(CommandLine) LIKE '%LoadDLL%'
            AND lower(CommandLine) LIKE '%.dll%'
        )
    )
'''
)
huntCase.show(10, 75)

### Change Default File Association To Executable
##### description : Detects when a program changes the default file association of any extension to an executable. When a file is opened, the default program used to open the file (also called the file association or handler) is checked. File association selections are stored in the Windows Registry and can be edited by users, administrators, or programs that have Registry access or by administrators using the built-in assoc utility. Applications can modify the file association for a given file extension to call an arbitrary program when a file with the given extension is opened.


In [ ]:
huntCase = spark.sql(
'''
SELECT computer_name, ParentImage, ParentCommandLine, Image, CommandLine, OriginalFileName, CurrentDirectory, User
FROM sparktable
WHERE (source_name = "Microsoft-Windows-Sysmon" AND event_id = 1
        AND
        (
            lower(CommandLine) LIKE '%cmd%'
            AND lower(CommandLine) LIKE '%/c%'
            AND lower(CommandLine) LIKE '%assoc%'

        )
    )
'''
)
huntCase.show(10, 75)

### Suspicious Firewall Change 
##### description : Detects when a firewall rule is modified or changed 

In [ ]:
huntCase = spark.sql(
'''
SELECT computer_name, ParentImage, ParentCommandLine, Image, CommandLine, OriginalFileName, CurrentDirectory, User
FROM sparktable
WHERE (source_name = "Microsoft-Windows-Sysmon" AND event_id = 1
        AND
        (
            lower(CommandLine) LIKE '%cmd%'
            AND lower(CommandLine) LIKE '% /c %'
            AND lower(CommandLine) LIKE '%advfirewall%'
        )

    )
'''
)
huntCase.show(10, 100)

### Windows Cmd Delete File
##### description :     Adversaries may delete files left behind by the actions of their intrusion activity. Malware, tools, or other non-native files dropped or created on a system by an adversary may leave traces to indicate to what was done within a network and how. Removal of these files can occur during an intrusion, or as part of a post-intrusion process to minimize the adversary's footprint.

In [ ]:
huntCase = spark.sql(
'''
SELECT computer_name, ParentImage, ParentCommandLine, Image, CommandLine, OriginalFileName, CurrentDirectory, User
FROM sparktable
WHERE (source_name = "Microsoft-Windows-Sysmon" AND event_id = 1
        AND
        (
            lower(CommandLine) LIKE '%del %'
            AND lower(CommandLine) LIKE '%/f%'
        )
        OR
        (
            lower(CommandLine) LIKE '%rmdir%'
            AND lower(CommandLine) LIKE '%/s%'
            AND lower(CommandLine) LIKE '%/q%'
        )

    )
'''
)
huntCase.show(10, 180)

### Suspicious Dosfuscation Character in Commandline
##### description :   Posssible Payload Obfuscation

In [ ]:
huntCase = spark.sql(
'''
SELECT computer_name, ParentImage, ParentCommandLine, Image, CommandLine, OriginalFileName, CurrentDirectory, User
FROM sparktable
WHERE (source_name = "Microsoft-Windows-Sysmon" AND event_id = 1
        AND
        (
            lower(CommandLine) LIKE '%^^%'
            OR lower(CommandLine) LIKE '%""%'
            OR lower(CommandLine) LIKE '%,;,%'
            OR lower(CommandLine) LIKE '%\%COMSPEC:~%'
            OR lower(CommandLine) LIKE '%s^et%'
            OR lower(CommandLine) LIKE '%s^e^t%'
            OR lower(CommandLine) LIKE '%se^t%'
            OR lower(CommandLine) LIKE '%se^t%'
            OR lower(CommandLine) LIKE '%\%\%%'
            OR lower(CommandLine) LIKE '%&&%'
        )

    )
'''
)
huntCase.show(20, 80)

### Suspicious CMD Shell Redirect
##### description :   Detects inline windows shell commands redirecting output via the ">" symbol to a suspicous location

In [ ]:
huntCase = spark.sql(
'''
SELECT computer_name, ParentImage, ParentCommandLine, Image, CommandLine, OriginalFileName, CurrentDirectory, User
FROM sparktable
WHERE (source_name = "Microsoft-Windows-Sysmon" AND event_id = 1
        AND
        (
            lower(Image) LIKE '%\\cmd.exe'
            AND 
            (
                lower(CommandLine) LIKE '%>%\%USERPROFILE%'
                OR lower(CommandLine) LIKE '%>%\%APPDATA%%'
                OR lower(CommandLine) LIKE '%\%TEMP:~%'
                OR lower(CommandLine) LIKE '%>\\Users\\Public\\%'
                OR lower(CommandLine) LIKE '%>%pipe%'
                OR lower(CommandLine) LIKE '%s^e^t%'
                OR lower(CommandLine) LIKE '%se^t%'
                OR lower(CommandLine) LIKE '%se^t%'
                OR lower(CommandLine) LIKE '%\%\%%'
                OR lower(CommandLine) LIKE '%&&%'
            )
        )

    )
'''
)
huntCase.show(10, 70)

### CMSTP Execution Process Creation
##### description :   Detects various indicators of Microsoft Connection Manager Profile Installer execution

In [ ]:
huntCase = spark.sql(
'''
SELECT computer_name, ParentImage, ParentCommandLine, Image, CommandLine, OriginalFileName, CurrentDirectory, User
FROM sparktable
WHERE (source_name = "Microsoft-Windows-Sysmon" AND event_id = 1
        AND
        (
            lower(ParentImage) LIKE '%\\cmstp.exe'
        )

    )
'''
)
huntCase.show(10, 90)

### Operator Bloopers Cobalt Strike Commands
##### description :  Detects use of Cobalt Strike commands accidentally entered in the CMD shell

In [ ]:
huntCase = spark.sql(
'''
SELECT computer_name, ParentImage, ParentCommandLine, Image, CommandLine, OriginalFileName, CurrentDirectory, User
FROM sparktable
WHERE (source_name = "Microsoft-Windows-Sysmon" AND event_id = 1
        AND
        (
            (
                lower(Image) LIKE '%\\cmd.exe'
                OR lower(ParentImage) LIKE '%\\cmd.exe'
            )
            AND
            (
                lower(CommandLine) LIKE '%psinject%'
                OR lower(CommandLine) LIKE '%spawnas%'
                OR lower(CommandLine) LIKE '%make_token%'
                OR lower(CommandLine) LIKE '%remote-exec%'
                OR lower(CommandLine) LIKE '%rev2self%'
                OR lower(CommandLine) LIKE '%getsystem%'
                OR lower(CommandLine) LIKE '%logonpasswords%'
                OR lower(CommandLine) LIKE '%dcsync%'
                OR lower(CommandLine) LIKE '%psh%'
                OR lower(CommandLine) LIKE '%remote-exec%'
                OR lower(CommandLine) LIKE '%Invoke-UserHunter%'
                OR lower(CommandLine) LIKE '%Invoke-ShareFinder%'
                OR lower(CommandLine) LIKE '%Invoke-Kerberoast%'
                OR lower(CommandLine) LIKE '%Invoke-SMBAutoBrute%'
                OR lower(CommandLine) LIKE '%Invoke-Nightmare%'
                OR lower(CommandLine) LIKE '%zerologon%'
                OR lower(CommandLine) LIKE '%av_query%'
            )

        )

    )
'''
)
huntCase.show(10, 75)

### Suspicious Load by Rundll32
##### description :  Rundll32 can be use by Cobalt Strike with StartW function to load DLLs from the command line.

In [ ]:
huntCase = spark.sql(
'''
SELECT computer_name, ParentImage, ParentCommandLine, Image, CommandLine, CurrentDirectory, User
FROM sparktable
WHERE (source_name = "Microsoft-Windows-Sysmon" AND event_id = 1
        AND
        (
            (
                lower(OriginalFileName) LIKE '%rundll32.exe%'
                AND
                (
                    lower(CommandLine) LIKE '%.dll%'
                    OR lower(CommandLine) LIKE '%.cpl%'
                    OR lower(CommandLine) LIKE '%runhtmlapplication'
                    OR lower(CommandLine) LIKE '%sta'
                    OR lower(CommandLine) LIKE '%/sta'
                    OR lower(CommandLine) LIKE '%activexobject'
                    OR lower(CommandLine) LIKE '%wscript.shell'
                    OR lower(CommandLine) LIKE '%dllregisterserver'
                    OR lower(CommandLine) LIKE '%startw'
                    OR lower(CommandLine) LIKE '%launchapplication'
                    OR lower(CommandLine) LIKE '%.dll,a*/p:'
                    OR lower(CommandLine) LIKE '%installhinfsection'
                    OR lower(CommandLine) LIKE '%launchinfsection'
                    OR lower(CommandLine) LIKE '%runhtmlapplication'
                    OR lower(CommandLine) LIKE '%setupinfobjectinstallaction'
                )
                OR
                (
                   lower(CommandLine) LIKE '%url.dll%'
                   AND lower(CommandLine) LIKE '%openurl%'
                )
                OR
                (
                   lower(CommandLine) LIKE '%url.dll%'
                   AND lower(CommandLine) LIKE '%fileprotocolhandler%'
                )
            )
            AND NOT
            (
                lower(ParentCommandLine) LIKE '%svchost.exe -k%'
                OR lower(ParentCommandLine) LIKE '%ie4uinit.exe -cleariconcache%'
            )

        )

    )
'''
)
huntCase.show(100, 70)

### CobaltStrike Process Patterns
##### description :  Detects process patterns found in Cobalt Strike beacon activity (see reference for more details) and also cases in which a China Chopper like webshell is used to run whoami

In [ ]:
huntCase = spark.sql(
'''
SELECT computer_name, ParentImage, ParentCommandLine, Image, CommandLine, OriginalFileName, CurrentDirectory, User
FROM sparktable
WHERE (source_name = "Microsoft-Windows-Sysmon" AND event_id = 1
        AND
        (
            (
                lower(ParentCommandLine) LIKE '%>%.%pipe%'
                OR lower(ParentCommandLine) LIKE '%cmd.exe /c echo%'
            )
            OR
            (
                lower(CommandLine) LIKE '%>%.%pipe%'
                OR lower(CommandLine) LIKE '%cmd.exe /c echo%'
            )
            OR
                lower(ParentImage) LIKE '%\\runonce.exe%'
                AND lower(Image) LIKE '%cmd.exe%'

        )

    )
'''
)
huntCase.show(10, 70)

### Conti Ransomware Execution
##### description : Conti ransomware CommandLine TTP

In [ ]:
huntCase = spark.sql(
'''
SELECT computer_name, ParentImage, ParentCommandLine, Image, CommandLine, OriginalFileName, CurrentDirectory, User
FROM sparktable
WHERE (source_name = "Microsoft-Windows-Sysmon" AND event_id = 1
        AND
        (
            lower(CommandLine) LIKE '%-m %'
            AND  lower(CommandLine) LIKE '%-net %'
            AND lower(CommandLine) LIKE '%-size %'
            AND lower(CommandLine) LIKE '%-nomutex %'
            AND lower(CommandLine) LIKE '%-p %'
            AND lower(CommandLine) LIKE '%$%'
        )

    )
'''
)
huntCase.show(10, 70)

### Copying Sensitive Files with Credential Data
##### description : Files with well-known filenames (sensitive files with credential data) copying

In [ ]:
huntCase = spark.sql(
'''
SELECT computer_name, ParentImage, ParentCommandLine, Image, CommandLine, OriginalFileName, CurrentDirectory, User
FROM sparktable
WHERE (source_name = "Microsoft-Windows-Sysmon" AND event_id = 1
        AND
        (
            lower(Image) LIKE '%\\esentutl.exe%'
        )
        AND
        (
            lower(CommandLine) LIKE '%vss%'
            OR lower(CommandLine) LIKE '% /m %'
            OR lower(CommandLine) LIKE '% /y %'
            OR lower(CommandLine) LIKE '%windows%ntds%ntds.dit%'
            OR lower(CommandLine) LIKE '%config%sam%'
            OR lower(CommandLine) LIKE '%config%security%'
            OR lower(CommandLine) LIKE '%config%system %'
            OR lower(CommandLine) LIKE '%repair%sam%'
            OR lower(CommandLine) LIKE '%repair%system%'
            OR lower(CommandLine) LIKE '%repair%security%'
            OR lower(CommandLine) LIKE '%config%regback%sam%'
            OR lower(CommandLine) LIKE '%config%regback%system%'
            OR lower(CommandLine) LIKE '%config%regback%security%'
        )
    )
'''
)
huntCase.show(10, 80)

### CrackMapExec Process Patterns
##### description : Detects suspicious process patterns found in logs when CrackMapExec is used

In [ ]:
huntCase = spark.sql(
'''
SELECT computer_name, ParentImage, ParentCommandLine, Image, CommandLine, OriginalFileName, CurrentDirectory, User
FROM sparktable
WHERE (source_name = "Microsoft-Windows-Sysmon" AND event_id = 1
        AND
        (
            lower(CommandLine) LIKE '%cmd.exe /c%'
            AND lower(CommandLine) LIKE '%tasklist /fi%'
            AND lower(CommandLine) LIKE '%Imagename%lsass.exe%'
        )
        OR
        (
            lower(CommandLine) LIKE '%do%rundll32.exe%.dll,%'
            AND lower(CommandLine) LIKE '%\\temp\\%'
            AND lower(CommandLine) LIKE '%full%'
            AND lower(CommandLine) LIKE '%B%'
        )
    )
'''
)
huntCase.show(10, 70)

### Mavinject Inject DLL Into Running Process
##### description :  Injects arbitrary DLL into running process specified by process ID. Requires Windows 10.

In [ ]:
huntCase = spark.sql(
'''
SELECT computer_name, ParentImage, ParentCommandLine, Image, CommandLine, OriginalFileName, CurrentDirectory, User
FROM sparktable
WHERE (source_name = "Microsoft-Windows-Sysmon" AND event_id = 1
        AND
        (
            (
                lower(Image) LIKE '%\\mavinject32.exe%'
                OR lower(Image) LIKE '%\\mavinject64.exe%'
            )
            AND
            (
                lower(CommandLine) LIKE '%injectrunning%'
                AND lower(CommandLine) LIKE '%.dll%'
            )
        )
        OR
        (
            (
                lower(OriginalFileName) LIKE '%mavinject32%'
                OR lower(OriginalFileName) LIKE '%mavinject64%'
            )
            AND
            (
                lower(CommandLine) LIKE '%injectrunning%'
                AND lower(CommandLine) LIKE '%.dll%'
            )
        )
    )
'''
)
huntCase.show(10, 50)

### Dropping Of Password Filter DLL
##### description :  Detects dropping of dll files in system32 that may be used to retrieve user credentials from LSASS

In [ ]:
huntCase = spark.sql(
'''
SELECT computer_name, ParentImage, ParentCommandLine, Image, CommandLine, OriginalFileName, CurrentDirectory, User
FROM sparktable
WHERE (source_name = "Microsoft-Windows-Sysmon" AND event_id = 1
        AND
        (
           lower(CommandLine) LIKE '%hklm%system%currentcontrolset%control%'
           AND lower(CommandLine) LIKE '%scecli\0%'
           AND lower(CommandLine) LIKE '%reg%add%'
        )
    )
'''
)
huntCase.show(10, 70)

### Curl Usage on Windows
##### description :  Detects a curl process start on Windows, which indicates a file download from a remote location or a simple web request to a remote server

In [ ]:
huntCase = spark.sql(
'''
SELECT computer_name, ParentImage, ParentCommandLine, Image, CommandLine, OriginalFileName, CurrentDirectory, User
FROM sparktable
WHERE (source_name = "Microsoft-Windows-Sysmon" AND event_id = 1
        AND
        (
            lower(CommandLine) LIKE '%\\curl.exe%'
            OR lower(OriginalFileName) LIKE '%curl.exe%'
        )
    )
'''
)
huntCase.show(10, 70)

### CVE-2021-26857 Exchange Exploitation
##### description :  Detects possible successful exploitation for vulnerability described in CVE-2021-26857 by looking for abnormal subprocesses spawning by Exchange Server's Unified Messaging service

In [ ]:
huntCase = spark.sql(
'''
SELECT computer_name, ParentImage, ParentCommandLine, Image, CommandLine, OriginalFileName, CurrentDirectory, User
FROM sparktable
WHERE (source_name = "Microsoft-Windows-Sysmon" AND event_id = 1
        AND
        (
            lower(ParentImage) LIKE '%umworkerprocess.exe%'
        )
        AND NOT
        (
            lower(Image) LIKE '%wermgr.exe%'
            OR lower(Image) LIKE '%werfault.exe%'
        )
    )
'''
)
huntCase.show(10, 70)

### Data Compressed - rar.exe
##### description :  An adversary may compress data (e.g., sensitive documents) that is collected prior to exfiltration in order to make it portable and minimize the amount of data sent over the network.

In [ ]:
huntCase = spark.sql(
'''
SELECT computer_name, ParentImage, ParentCommandLine, Image, CommandLine, OriginalFileName, CurrentDirectory, User
FROM sparktable
WHERE (source_name = "Microsoft-Windows-Sysmon" AND event_id = 1
        AND
        (
            lower(Image) LIKE '%\\rar.exe%'
            AND lower(CommandLine) LIKE '% a %'
        )
    )
'''
)
huntCase.show(10, 70)

### Wbadmin Delete Systemstatebackup
##### description :    Deletes the Windows systemstatebackup using wbadmin.exe. This technique is used by numerous ransomware families. This may only be successful on server platforms that have Windows Backup enabled.

In [ ]:
huntCase = spark.sql(
'''
SELECT computer_name, ParentImage, ParentCommandLine, Image, CommandLine, OriginalFileName, CurrentDirectory, User
FROM sparktable
WHERE (source_name = "Microsoft-Windows-Sysmon" AND event_id = 1
        AND
        (
            (
                lower(Image) LIKE '%\\wbadmin.exe%'
                OR  lower(CommandLine) LIKE '%wbadmin%'
            )
            AND
            (
                lower(CommandLine) LIKE '%delete %'
                AND lower(CommandLine) LIKE '%systemstatebackup %'
                AND lower(CommandLine) LIKE '%-keepversions:0%'
            )
        )
    )
'''
)
huntCase.show(10, 80)

### Detecting Fake Instances Of Hxtsr.exe
##### description : HxTsr.exe is a Microsoft compressed executable file called Microsoft Outlook Communications.HxTsr.exe is part of Outlook apps, because it resides in a hidden "WindowsApps" subfolder of "C:\Program Files". Its path includes a version number, e.g., "C:\Program Files\WindowsApps\microsoft.windowscommunicationsapps_17.7466.41167.0_x64__8wekyb3d8bbwe\HxTsr.exe". Any instances of hxtsr.exe not in this folder may be malware camouflaging itself as HxTsr.exe

In [ ]:
huntCase = spark.sql(
'''
SELECT computer_name, ParentImage, ParentCommandLine, Image, CommandLine, OriginalFileName, CurrentDirectory, User
FROM sparktable
WHERE (source_name = "Microsoft-Windows-Sysmon" AND event_id = 1
        AND
        (
            lower(Image) LIKE '%hxtsr.exe%'
            AND NOT
                (
                    lower(CurrentDirectory) LIKE '%program files\\windowsapps\\microsoft.windowscommunicationsapps_%'
                    OR lower(CurrentDirectory) LIKE '%\\hxtsr.exe%'
                )
        )
    )
'''
)
huntCase.show(10, 80)

### DInject PowerShell Cradle CommandLine Flags
##### description : Detects the use of the Dinject PowerShell cradle based on the specific flags

In [ ]:
huntCase = spark.sql(
'''
SELECT computer_name, ParentImage, ParentCommandLine, Image, CommandLine, OriginalFileName, CurrentDirectory, User
FROM sparktable
WHERE (source_name = "Microsoft-Windows-Sysmon" AND event_id = 1
        AND
        (
            lower(CommandLine) LIKE '% /am51%'
            OR lower(CommandLine) LIKE '% /password%'
        )
        AND NOT
                (
                    lower(CommandLine) LIKE '% /passwordchg%'
                )
    )
'''
)
huntCase.show(10, 80)

###  DNS ServerLevelPluginDll Install
##### description : Detects the installation of a plugin DLL via ServerLevelPluginDll parameter in Registry, which can be used to execute code in context of the DNS server (restart required)

In [ ]:
huntCase = spark.sql(
'''
SELECT computer_name, ParentImage, ParentCommandLine, Image, CommandLine, OriginalFileName, CurrentDirectory, User
FROM sparktable
WHERE (source_name = "Microsoft-Windows-Sysmon" AND event_id = 1
        AND
        (
            lower(Image) LIKE '%\\dnscmd.exe%'
        )
        AND
        (
            lower(CommandLine) LIKE '%/config%'
            OR lower(CommandLine) LIKE '%/serverlevelplugindll%'
        )
    )
'''
)
huntCase.show(10, 80)

###  DNSCat2 Powershell Implementation Detection Via Process Creation
##### description : The PowerShell implementation of DNSCat2 calls nslookup to craft queries. Counting nslookup processes spawned by PowerShell will show hundreds or thousands of instances if PS DNSCat2 is active locally.

In [ ]:
huntCase = spark.sql(
'''
SELECT computer_name, ParentImage, ParentCommandLine, Image, CommandLine, OriginalFileName, CurrentDirectory, User
FROM sparktable
WHERE (source_name = "Microsoft-Windows-Sysmon" AND event_id = 1
        AND
        (
            lower(ParentImage) LIKE '%\\powershell.exe%'
            OR lower(ParentImage) LIKE '%\\pwsh.exe%'
        )
        AND
        (
            lower(Image) LIKE '%\\nslookup.exe%'
        )
    )
'''
)
huntCase.show(10, 80)

###   Dotnet.exe Exec Dll and Execute Unsigned Code LOLBIN
##### description : dotnet.exe will execute any DLL and execute unsigned code

In [ ]:
huntCase = spark.sql(
'''
SELECT computer_name, ParentImage, ParentCommandLine, Image, CommandLine, OriginalFileName, CurrentDirectory, User
FROM sparktable
WHERE (source_name = "Microsoft-Windows-Sysmon" AND event_id = 1
        AND
        (
            lower(Image) LIKE '%\\dotnet.exe%'
        )
        AND
        (
            lower(CommandLine) LIKE '%.dll%'
            AND lower(CommandLine) LIKE '%.csproj%'
        )
    )
'''
)
huntCase.show(10, 80)

###   Abusing Permissions Using Dsacls
##### description : Detects usage of Dsacls to grant over permissive permissions

In [ ]:
huntCase = spark.sql(
'''
SELECT computer_name, ParentImage, ParentCommandLine, Image, CommandLine, OriginalFileName, CurrentDirectory, User
FROM sparktable
WHERE (source_name = "Microsoft-Windows-Sysmon" AND event_id = 1
        AND
        (
            lower(Image) LIKE '%\\dsacls.exe%'
            AND lower(CommandLine) LIKE '% /G %'
        )
        AND
        (
            lower(CommandLine) LIKE '%gr%'
            OR lower(CommandLine) LIKE '%ge%'
            OR lower(CommandLine) LIKE '%gw%'
            OR lower(CommandLine) LIKE '%ga%'
            OR lower(CommandLine) LIKE '%wp%'
            OR lower(CommandLine) LIKE '%wd%'
        )
    )
'''
)
huntCase.show(10, 80)

###   Password Spraying Attempts Using Dsacls
##### description : Detects possible password spraying attempts using Dsacls

In [ ]:
huntCase = spark.sql(
'''
SELECT computer_name, ParentImage, ParentCommandLine, Image, CommandLine, OriginalFileName, CurrentDirectory, User
FROM sparktable
WHERE (source_name = "Microsoft-Windows-Sysmon" AND event_id = 1
        AND
        (
            lower(Image) LIKE '%\\dsacls.exe%'
        )
        AND
        (
            lower(CommandLine) LIKE '%/user:%'
            AND lower(CommandLine) LIKE '%/passwd%'
        )
    )
'''
)
huntCase.show(10, 80)

###   DumpStack.log Defender Evasion
##### description : Detects the use of the filename DumpStack.log to evade Microsoft Defender

In [ ]:
huntCase = spark.sql(
'''
SELECT computer_name, ParentImage, ParentCommandLine, Image, CommandLine, OriginalFileName, CurrentDirectory, User
FROM sparktable
WHERE (source_name = "Microsoft-Windows-Sysmon" AND event_id = 1
        AND
        (
            lower(Image) LIKE '%\\dumpstack.log%'
        )
        AND
        (
            lower(CommandLine) LIKE '% -o dumpstack.log%'
        )
    )
'''
)
huntCase.show(10, 80)

###  Hidden Powershell in Link File Pattern
##### description : Detects events that appear when a user click on a link file with a powershell command in it

In [ ]:
huntCase = spark.sql(
'''
SELECT computer_name, ParentImage, ParentCommandLine, Image, CommandLine, OriginalFileName, CurrentDirectory, User
FROM sparktable
WHERE (source_name = "Microsoft-Windows-Sysmon" AND event_id = 1
        AND
        (
            lower(ParentImage) LIKE '%windows\\explorer.exe%'
            OR lower(Image) LIKE '%cmd.exe%'
        )
        AND
        (
             lower(CommandLine) LIKE '%powershell%'
             AND lower(CommandLine) LIKE '%.lnk%'
        )
    )
'''
)
huntCase.show(10, 110)

###  Encoded FromBase64String
##### description :  Detects a base64 encoded FromBase64String keyword in a process command line

In [ ]:
huntCase = spark.sql(
'''
SELECT computer_name, ParentImage, ParentCommandLine, Image, CommandLine, OriginalFileName, CurrentDirectory, User
FROM sparktable
WHERE (source_name = "Microsoft-Windows-Sysmon" AND event_id = 1
        AND
        (
            lower(CommandLine) LIKE '%frombase64string%'
            OR lower(CommandLine) LIKE '%decode%'
            OR lower(CommandLine) LIKE '%gzip%'
        )
        OR
        (
            lower(ParentCommandLine) LIKE '%frombase64string%'
            OR lower(ParentCommandLine) LIKE '%decode%'
            OR lower(ParentCommandLine) LIKE '%gzip%'
        )
        OR
        (
            CommandLine LIKE '%OgA6AEYAcgBvAG0AQgBhAHMAZQA2ADQAUwB0AHIAaQBuAGcA%'
            OR CommandLine LIKE '%oAOgBGAHIAbwBtAEIAYQBzAGUANgA0AFMAdAByAGkAbgBnA%'
            OR CommandLine LIKE '%6ADoARgByAG8AbQBCAGEAcwBlADYANABTAHQAcgBpAG4AZw%'
            OR CommandLine LIKE '%^%^%^%'
        )
    )
'''
)
huntCase.show(10, 50)

###  Encoded IEX
##### description :  Detects a base64 encoded IEX command string in a process command line

In [ ]:
huntCase = spark.sql(
'''
SELECT computer_name, ParentImage, ParentCommandLine, Image, CommandLine, OriginalFileName, CurrentDirectory, User
FROM sparktable
WHERE (source_name = "Microsoft-Windows-Sysmon" AND event_id = 1
        AND
        (
            lower(CommandLine) LIKE '%iex ([%'
            OR lower(CommandLine) LIKE '%iex (new%'
        )
        OR
        (
            CommandLine LIKE '%SQBFAFgAIAAoAFsA%'
            OR CommandLine LIKE '%kARQBYACAAKABbA%'
            OR CommandLine LIKE '%JAEUAWAAgACgAWw%'
            OR CommandLine LIKE '%aQBlAHgAIAAoAFsA%'
            OR CommandLine LIKE '%kAZQB4ACAAKABbA%'
            OR CommandLine LIKE '%pAGUAeAAgACgAWw%'
            OR CommandLine LIKE '%aQBlAHgAIAAoAE4AZQB3A%'
            OR CommandLine LIKE '%kAZQB4ACAAKABOAGUAdw%'
            OR CommandLine LIKE '%pAGUAeAAgACgATgBlAHcA%'
            OR CommandLine LIKE '%SQBFAFgAIAAoAE4AZQB3A%'
            OR CommandLine LIKE '%kARQBYACAAKABOAGUAdw%'
            OR CommandLine LIKE '%JAEUAWAAgACgATgBlAHcA%'
        )
    )
'''
)
huntCase.show(10, 50)

###  Enumeration for Credentials in RegistryEnumeration for Credentials in Registry
##### description :  Detects processes that query known 3rd party registry keys that holds credentials via commandline.  Adversaries may search the Registry on compromised systems for insecurely stored credentials. The Windows Registry stores configuration information that can be used by the system or other programs. Adversaries may query the Registry looking for credentials and passwords that have been stored for use by other programs or services Adversaries may search the Registry on compromised systems for insecurely stored credentials. The Windows Registry stores configuration information that can be used by the system or other programs. Adversaries may query the Registry looking for credentials and passwords that have been stored for use by other programs or services

In [ ]:
huntCase = spark.sql(
'''
SELECT computer_name, ParentImage, ParentCommandLine, Image, CommandLine, OriginalFileName, CurrentDirectory, User
FROM sparktable
WHERE (source_name = "Microsoft-Windows-Sysmon" AND event_id = 1
        AND
        (
            lower(CommandLine) LIKE '%software%simontatham%putty%sessions%'
            OR lower(CommandLine) LIKE '%software%simontatham%putty%sshhostkeys%'
            OR lower(CommandLine) LIKE '%software\\mobatek\\mobaxterm\\%'
            OR lower(CommandLine) LIKE '%software\\wow6432node\\radmin\\v3.0\\server\\parameters\\radmin%'
            OR lower(CommandLine) LIKE '%software\\aerofox\\foxmailpreview%'
            OR lower(CommandLine) LIKE '%software\\aerofox\\foxmail\\v3.1%'
            OR lower(CommandLine) LIKE '%software\\incredimail\\identities%'
            OR lower(CommandLine) LIKE '%software\\qualcomm\\eudora\\commandline%'
            OR lower(CommandLine) LIKE '%software\\rimarts\\b2\\settings%'
            OR lower(CommandLine) LIKE '%software\\openvpn-gui\\configs%'
            OR lower(CommandLine) LIKE '%software\\martin prikryl\\winscp 2\\sessions%'
            OR lower(CommandLine) LIKE '%software\\ftpware\\coreftp\\sites%'
            OR lower(CommandLine) LIKE '%software\\downloadmanager\\passwords%'
            OR lower(CommandLine) LIKE '%software\\openssh\\agent\\keys%'
            OR lower(CommandLine) LIKE '%software\\tightvnc\\server%'
            OR lower(CommandLine) LIKE '%software\\orl\\winvnc3\\password%'
            OR lower(CommandLine) LIKE '%software\\realvnc\\winvnc4%'
        )
        OR
        (
            (
                lower(CommandLine) LIKE '% query %'
                AND lower(CommandLine) LIKE '%/t %'
                AND lower(CommandLine) LIKE '%reg_sz%'
                AND lower(CommandLine) LIKE '%/s %'
            )
            AND
            (
                (
                    lower(CommandLine) LIKE '%hkcu%' 
                    AND lower(CommandLine) LIKE '% /f %'
                )
                OR
                (
                    lower(CommandLine) LIKE '%hklm%' 
                    AND lower(CommandLine) LIKE '% /f %'
                )
            )
        )
    )
'''
)
huntCase.show(10, 70)

###  COMPlus_ETWEnabled Command Line Arguments
##### description :  Potential adversaries stopping ETW providers recording loaded .NET assemblies.

In [ ]:
huntCase = spark.sql(
'''
SELECT computer_name, ParentImage, ParentCommandLine, Image, CommandLine, OriginalFileName, CurrentDirectory, User
FROM sparktable
WHERE (source_name = "Microsoft-Windows-Sysmon" AND event_id = 1
        AND
        (
            lower(CommandLine) LIKE '%complus_etwenabled=0%'
        )
    )
'''
)
huntCase.show(10, 70)

###  Disable of ETW Trace
##### description :  Detects a command that clears or disables any ETW trace log which could indicate a logging evasion.

In [ ]:
huntCase = spark.sql(
'''
SELECT computer_name, ParentImage, ParentCommandLine, Image, CommandLine, OriginalFileName, CurrentDirectory, User
FROM sparktable
WHERE (source_name = "Microsoft-Windows-Sysmon" AND event_id = 1
        AND
        (
            lower(CommandLine) LIKE '%cl%'
            AND lower(CommandLine) LIKE '%trace%'
        )
        OR
        (
            lower(CommandLine) LIKE '%clear-log%'
            AND lower(CommandLine) LIKE '%trace%'
        )
        OR
        (
            lower(CommandLine) LIKE '%sl%'
            AND lower(CommandLine) LIKE '%/e:false%'
        )
        OR
        (
            lower(CommandLine) LIKE '%set-log%'
            AND lower(CommandLine) LIKE '%/e:false%'
        )
        OR
        (
            lower(CommandLine) LIKE '%logman%'
            AND lower(CommandLine) LIKE '%update%'
            AND lower(CommandLine) LIKE '%trace%'
            AND lower(CommandLine) LIKE '%--p%'
            AND lower(CommandLine) LIKE '%-ets%'
        )
        OR
        (
            lower(CommandLine) LIKE '%remove-etwtraceprovider%'
        )
        OR
        (
            lower(CommandLine) LIKE '%set-etwtraceprovider%'
            AND lower(CommandLine) LIKE '%0x11%'
        )
    )
'''
)
huntCase.show(10, 70)

###  WinRM Access with Evil-WinRM
##### description :  Adversaries may use Valid Accounts to log into a computer using the Remote Desktop Protocol (RDP). The adversary may then perform actions as the logged-on user.

In [ ]:
huntCase = spark.sql(
'''
SELECT computer_name, ParentImage, ParentCommandLine, Image, CommandLine, OriginalFileName, CurrentDirectory, User
FROM sparktable
WHERE (source_name = "Microsoft-Windows-Sysmon" AND event_id = 1
        AND
        (
            lower(Image) LIKE '%\\ruby.exe%'
        )
        AND
        (
            lower(CommandLine) LIKE '%-i %'
            OR lower(CommandLine) LIKE '%-u %'
            OR lower(CommandLine) LIKE '%-p %'
        )
    )
'''
)
huntCase.show(10, 70)

###  Exfiltration and Tunneling Tools Execution
##### description : Execution of well known tools for data exfiltration and tunneling

In [ ]:
huntCase = spark.sql(
'''
SELECT computer_name, ParentImage, ParentCommandLine, Image, CommandLine, OriginalFileName, CurrentDirectory, User
FROM sparktable
WHERE (source_name = "Microsoft-Windows-Sysmon" AND event_id = 1
        AND
        (
            lower(Image) LIKE '%\\plink.exe%'
            OR lower(Image) LIKE '%\\socat.exe%'
            OR lower(Image) LIKE '%\\stunnel.exe%'
            OR lower(Image) LIKE '%\\httptunnel.exe%'
        )
    )
'''
)
huntCase.show(10, 70)

###  Droppers Exploiting CVE-2017-11882
##### description : Detects exploits that use CVE-2017-11882 to start EQNEDT32.EXE and other sub processes like mshta.exe

In [ ]:
huntCase = spark.sql(
'''
SELECT computer_name, ParentImage, ParentCommandLine, Image, CommandLine, OriginalFileName, CurrentDirectory, User
FROM sparktable
WHERE (source_name = "Microsoft-Windows-Sysmon" AND event_id = 1
        AND
        (
            lower(ParentImage) LIKE '%\\eqnedt32.exe%'
            OR lower(Image) LIKE '%\\eqnedt32.exe%'
        )
    )
'''
)
huntCase.show(10, 70)

###  Exploit for CVE-2017-8759
##### description : Detects Winword starting uncommon sub process csc.exe as used in exploits for CVE-2017-8759; has been modified to look for suspicious lolbins

In [ ]:
huntCase = spark.sql(
'''
SELECT computer_name, ParentImage, ParentCommandLine, Image, CommandLine, OriginalFileName, CurrentDirectory, User
FROM sparktable
WHERE (source_name = "Microsoft-Windows-Sysmon" AND event_id = 1
        AND
        (
            lower(ParentImage) LIKE '%\\winword.exe%'
            AND
            (
                lower(Image) LIKE '%\\csc.exe%'
                OR lower(Image) LIKE '%\\werfault.exe%'
                OR lower(Image) LIKE '%\\mshta.exe%'
                OR lower(Image) LIKE '%\\rundll32.exe%'
                OR lower(Image) LIKE '%\\cmd.exe%'
                OR lower(Image) LIKE '%\\powershell.exe%'
                OR lower(Image) LIKE '%\\winword.exe%'
            )
        )
    )
'''
)
huntCase.show(10, 70)

###  Exploiting SetupComplete.cmd CVE-2019-1378
##### description : Detects exploitation attempt of privilege escalation vulnerability via SetupComplete.cmd and PartnerSetupComplete.cmd described in CVE-2019-1378

In [ ]:
huntCase = spark.sql(
'''
SELECT computer_name, ParentImage, ParentCommandLine, Image, CommandLine, OriginalFileName, CurrentDirectory, User
FROM sparktable
WHERE (source_name = "Microsoft-Windows-Sysmon" AND event_id = 1
        AND
        (
            lower(ParentCommandLine) LIKE '%\\cmd.exe%'
            AND lower(ParentCommandLine) LIKE '%/c%'
            AND lower(ParentCommandLine) LIKE '%windows%setup%scripts%'
            AND lower(ParentCommandLine) LIKE '%setupcomplete.cmd%'
            AND lower(ParentCommandLine) LIKE '%partnersetupcomplete.cmd%'
        )
    )
'''
)
huntCase.show(10, 70)

###   Exploiting CVE-2019-1388
##### description :  Detects an exploitation attempt in which the UAC consent dialogue is used to invoke an Internet Explorer process running as LOCAL_SYSTEM

In [ ]:
huntCase = spark.sql(
'''
SELECT computer_name, ParentImage, ParentCommandLine, Image, CommandLine, OriginalFileName, CurrentDirectory, User
FROM sparktable
WHERE (source_name = "Microsoft-Windows-Sysmon" AND event_id = 1
        AND
        (
            lower(ParentImage) LIKE '%\\consent.exe%'
            AND lower(Image) LIKE '%\\iexplore.exe%'
            AND lower(CommandLine) LIKE '%http%'
        )
        AND
        (
            lower(User) LIKE '%authori%'
            OR lower(User) LIKE '%autori%'
        )
    )
'''
)
huntCase.show(10, 70)

###  Exploited CVE-2020-10189 Zoho ManageEngine
##### description : Detects the exploitation of Zoho ManageEngine Desktop Central Java Deserialization vulnerability reported as CVE-2020-10189

In [ ]:
huntCase = spark.sql(
'''
SELECT computer_name, ParentImage, ParentCommandLine, Image, CommandLine, OriginalFileName, CurrentDirectory, User
FROM sparktable
WHERE (source_name = "Microsoft-Windows-Sysmon" AND event_id = 1
        AND
        (
            lower(ParentImage) LIKE '%desktopcentral_server\\jre\\bin\\java.exe%'
        )
        AND
        (
            lower(Image) LIKE '%\\cmd.exe%'
            OR lower(Image) LIKE '%\\powershell.exe%'
            OR lower(Image) LIKE '%\\pwsh.exe%'
            OR lower(Image) LIKE '%\\bitsadmin.exe%'
        )
    )
'''
)
huntCase.show(10, 70)

###  DNS RCE CVE-2020-1350
##### description :  Detects exploitation of DNS RCE bug reported in CVE-2020-1350 by the detection of suspicious sub process

In [ ]:
huntCase = spark.sql(
'''
SELECT computer_name, ParentImage, ParentCommandLine, Image, CommandLine, OriginalFileName, CurrentDirectory, User
FROM sparktable
WHERE (source_name = "Microsoft-Windows-Sysmon" AND event_id = 1
        AND
        (
            lower(ParentImage) LIKE '%system32\\dns.exe%'
        )
        AND
        (
            lower(Image) LIKE '%system32\\werfault.exe%'
            OR lower(Image) LIKE '%system32\\conhost.exe%'
            OR lower(Image) LIKE '%system32\\dnscmd.exe%'
            OR lower(Image) LIKE '%system32\\dns.exe%'
        )
    )
'''
)
huntCase.show(10, 70)

###  SystemNightmare Exploitation Script Execution
##### description : Detects the exploitation of PrinterNightmare to get a shell as LOCAL_SYSTEM

In [ ]:
huntCase = spark.sql(
'''
SELECT computer_name, ParentImage, ParentCommandLine, Image, CommandLine, OriginalFileName, CurrentDirectory, User
FROM sparktable
WHERE (source_name = "Microsoft-Windows-Sysmon" AND event_id = 1
        AND
        (
            lower(CommandLine) LIKE '%printnightmare.gentilkiwi.com%'
            OR lower(CommandLine) LIKE '%user:gentilguest%'
            OR lower(CommandLine) LIKE '%kiwi kegit printer%'
        )
    )
'''
)
huntCase.show(10, 70)

###  File or Folder Permissions Modifications
##### description :  Detects a file or folder's permissions being modified.

In [ ]:
huntCase = spark.sql(
'''
SELECT computer_name, ParentImage, ParentCommandLine, Image, CommandLine, OriginalFileName, CurrentDirectory, User
FROM sparktable
WHERE (source_name = "Microsoft-Windows-Sysmon" AND event_id = 1
        AND
            lower(CommandLine) LIKE '%/grant%'
        AND
        (
            lower(Image) LIKE '%\\takeown.exe%'
            OR lower(Image) LIKE '%\\cacls.exe%'
            OR lower(Image) LIKE '%\\icacls.exe%'
            OR lower(Image) LIKE '%\\net.exe%'
            OR lower(Image) LIKE '%\\net1.exe%'
        )
        OR
        (
            lower(Image) LIKE '%\\attrib.exe%'
            AND
            (
                lower(CommandLine) LIKE '%-r%'
            )
        )
        AND NOT
        (
            lower(CommandLine) LIKE '%icacls%config\\connectivity.history /reset%'
            OR lower(CommandLine) LIKE '%appdata\\local\\programs\\microsoft vs code%'
        )
    )
'''
)
huntCase.show(10, 100)

###  Findstr GPP Passwords
##### description :   Look for the encrypted cpassword value within Group Policy Preference files on the Domain Controller. This value can be decrypted with gpp-decrypt.

In [ ]:
huntCase = spark.sql(
'''
SELECT computer_name, ParentImage, ParentCommandLine, Image, CommandLine, OriginalFileName, CurrentDirectory, User
FROM sparktable
WHERE (source_name = "Microsoft-Windows-Sysmon" AND event_id = 1
        AND
            lower(Image) LIKE '%\\findstr.exe%'
        AND
        (
            lower(CommandLine) LIKE '%cpassword%'
            AND lower(CommandLine) LIKE '%\\sysvol\\%'
            AND lower(CommandLine) LIKE '%.xml%'
        )
    )
'''
)
huntCase.show(10, 100)

###  ADCSPwn Hack Tool
##### description : Detects command line parameters used by ADCSPwn, a tool to escalate privileges in an active directory network by coercing authenticate from machine accounts and relaying to the certificate service

In [ ]:
huntCase = spark.sql(
'''
SELECT computer_name, ParentImage, ParentCommandLine, Image, CommandLine, OriginalFileName, CurrentDirectory, User
FROM sparktable
WHERE (source_name = "Microsoft-Windows-Sysmon" AND event_id = 1
        AND
        (
            lower(CommandLine) LIKE '% --adcs %'
            OR lower(CommandLine) LIKE '% --port %'
        )
    )
'''
)
huntCase.show(10, 100)

###  Bloodhound and Sharphound Hack Tool
##### description : Detects command line parameters used by Bloodhound and Sharphound hack tools

In [ ]:
huntCase = spark.sql(
'''
SELECT computer_name, ParentImage, ParentCommandLine, Image, CommandLine, OriginalFileName, CurrentDirectory, User
FROM sparktable
WHERE (source_name = "Microsoft-Windows-Sysmon" AND event_id = 1
        AND
        (
            lower(Image) LIKE '%\\bloodhound.exe%'
            OR lower(Image) LIKE '%\\sharphound.exe%'
        )
        OR
        (
            lower(CommandLine) LIKE '%-collectionmethod all%'
            OR lower(CommandLine) LIKE '%.exe -c all -d %'
            OR lower(CommandLine) LIKE '%invoke-bloodhound%'
            OR lower(CommandLine) LIKE '%invoke-sharphound%'
            OR lower(CommandLine) LIKE '%get-bloodhounddata%'
        )
        OR
        (
            lower(CommandLine) LIKE '%jsonfolder%'
            OR lower(CommandLine) LIKE '%-zipfilename%'
        )
        OR
        (
            lower(CommandLine) LIKE '%dconly%'
            AND lower(CommandLine) LIKE '%--nosavecache%'
        )
    )
'''
)
huntCase.show(10, 55)

###  Koadic Execution
##### description : Detects command line parameters used by Koadic hack tool

In [ ]:
huntCase = spark.sql(
'''
SELECT computer_name, ParentImage, ParentCommandLine, Image, CommandLine, OriginalFileName, CurrentDirectory, User
FROM sparktable
WHERE (source_name = "Microsoft-Windows-Sysmon" AND event_id = 1
        AND
        (
            lower(Image) LIKE '%\\cmd.exe%'
        )
        AND
        (
            lower(CommandLine) LIKE '%/q%'
            AND lower(CommandLine) LIKE '%/c%'
            AND lower(CommandLine) LIKE '%chcp%'
        )
    )
'''
)
huntCase.show(10, 80)

###  KrbRelay Hack Tool
##### description : Detects the use of KrbRelay, a Kerberos relaying tool

In [ ]:
huntCase = spark.sql(
'''
SELECT computer_name, ParentImage, ParentCommandLine, Image, CommandLine, OriginalFileName, CurrentDirectory, User
FROM sparktable
WHERE (source_name = "Microsoft-Windows-Sysmon" AND event_id = 1
        AND
        (
            lower(CommandLine) LIKE '%asreproast%'
            OR lower(CommandLine) LIKE '%dump%service:krbtgt%'
            OR lower(CommandLine) LIKE '%kerberoast%'
            OR lower(CommandLine) LIKE '%createnetonly%program:%'
            OR lower(CommandLine) LIKE '%ptt%ticket:%'
            OR lower(CommandLine) LIKE '%impersonateuser:%'
            OR lower(CommandLine) LIKE '%renew%ticket:%'
            OR lower(CommandLine) LIKE '%asktgt%user:%'
            OR lower(CommandLine) LIKE '%harvest%interval:%'
            OR lower(CommandLine) LIKE '%s4u%user:%'
            OR lower(CommandLine) LIKE '%s4u%ticket:%'
            OR lower(CommandLine) LIKE '%hash%password:%'
        )
    )
'''
)
huntCase.show(10, 80)

###  Domain hash retrieval via NTDS.dit detection
##### description : Detects the usage of vssadmin to access or copy NTDS.dit for domain hash retrieval

In [ ]:
huntCase = spark.sql(
'''
SELECT computer_name, ParentImage, ParentCommandLine, Image, CommandLine, OriginalFileName, CurrentDirectory, User
FROM sparktable
WHERE (source_name = "Microsoft-Windows-Sysmon" AND event_id = 1
        AND
            lower(CommandLine) LIKE '%vssadmin%'
        AND
        (
            lower(CommandLine) LIKE '%for%'
            OR lower(CommandLine) LIKE '%ntds%'
            OR lower(CommandLine) LIKE '%sam%'
            OR lower(CommandLine) LIKE '%resize%'
            OR lower(CommandLine) LIKE '%hklm%'
        )
        AND NOT
        (
            lower(CommandLine) LIKE '%add%'
        )
    )
'''
)
huntCase.show(10, 65)

###  LethalHTA detection
##### description : Detects the remote execution of Mshta via DCOM - LethalHTA

In [ ]:
huntCase = spark.sql(
'''
SELECT  computer_name, ParentImage, ParentCommandLine, Image, CommandLine, OriginalFileName, CurrentDirectory, User
FROM sparktable
WHERE (source_name = "Microsoft-Windows-Sysmon" AND event_id = 1
        AND
        (
            lower(ParentImage) LIKE '%svchost%'
            AND lower(CommandLine) LIKE '%mshta'
        )
    )
'''
)
huntCase.show(10, 60)

###  CMD or PowerShell spawned from MS Office Process
##### description : Detects the execution of command prompt or powershell from Microsoft Office Process

In [ ]:
huntCase = spark.sql(
'''
SELECT computer_name, ParentImage, ParentCommandLine, Image, CommandLine, OriginalFileName, CurrentDirectory, User
FROM sparktable
WHERE (source_name = "Microsoft-Windows-Sysmon" AND event_id = 1
        AND
        (
            lower(ParentImage) LIKE '%winword.exe'
            OR lower(ParentImage) LIKE '%excel.exe'
            OR lower(ParentImage) LIKE '%eqnedt32.exe'
            OR lower(ParentImage) LIKE '%msaccess.exe'
            OR lower(ParentImage) LIKE '%mspub.exe'
            OR lower(ParentImage) LIKE '%powerpnt.exe'
            OR lower(ParentImage) LIKE '%visio.exe'
        )
        AND
        (
            lower(Description) LIKE '%forfiles - executes a command on selected files%'
            OR lower(Description) LIKE '%microsoft (r) html application host%'
            OR lower(Description) LIKE '%microsoft ® console based script host%'
            OR lower(Description) LIKE '%microsoft ® windows based script host%'
            OR lower(Description) LIKE '%microsoft(c) register server%'
            OR lower(Description) LIKE '%microsoft® html help executable%'
            OR lower(Description) LIKE '%msbuild.exe%'
            OR lower(Description) LIKE '%network command shell%'
            OR lower(Description) LIKE '%task scheduler configuration tool%'
            OR lower(Description) LIKE '%windows command processor%'
            OR lower(Description) LIKE '%windows host process (rundll32)%'
            OR lower(Description) LIKE '%windows powershell%'
            OR lower(Description) LIKE '%windows® installer%'
            OR lower(Description) LIKE '%wmi commandline utility%'
            OR lower(Description) LIKE '%wmi standard event consumer - scripting%'
        )
    )
'''
)
huntCase.show(10, 60)

###  Commands Related to Credential Dumping Detected
##### description : Detects the Commands Related to Credential Dumping

In [ ]:
huntCase = spark.sql(
'''
SELECT computer_name, ParentImage, ParentCommandLine, Image, CommandLine, User
FROM sparktable
WHERE (source_name = "Microsoft-Windows-Sysmon" AND event_id = 1
        AND
        (
            lower(CommandLine) LIKE '%mimikatz%'
            OR lower(CommandLine) LIKE '%mimilib%'
            OR lower(CommandLine) LIKE '%sekurlsa::logonpasswords%'
            OR lower(CommandLine) LIKE '%lsadump::sam%'
            OR lower(CommandLine) LIKE '%mimidrv.sys%'
            OR lower(CommandLine) LIKE '%crypto::certificates%'
            OR lower(CommandLine) LIKE '%kerberos::golden%'
            OR lower(CommandLine) LIKE '%kerberos::list%'
            OR lower(CommandLine) LIKE '%kerberos::ptt%'
            OR lower(CommandLine) LIKE '%lsadump::dcsync%'
            OR lower(CommandLine) LIKE '%lsadump::lsa%'
            OR lower(CommandLine) LIKE '%lsadump::secrets%'
            OR lower(CommandLine) LIKE '%token::elevate%'
            OR lower(CommandLine) LIKE '%token::list%'
        )
        OR
        (
            lower(CommandLine) LIKE '%procdump.exe%'
            AND lower(CommandLine) LIKE '%-ma%'
        )
        OR
        (
            lower(CommandLine) LIKE '%rdrleakdiag.exe%'
            AND lower(CommandLine) LIKE '%fullmemdmp%'
        )
        OR
        (
            lower(CommandLine) LIKE '%tttracer.exe%'
            AND lower(CommandLine) LIKE '%-dumpFull%'
        )
        OR
        (
            lower(OriginalFileName) LIKE '%processdump.exe%'
            OR lower(OriginalFileName) LIKE '%writeminidump.exe%'
        )
        OR
        (
            lower(CommandLine) LIKE '%hacktool%'
            OR lower(CommandLine) LIKE '%rubeus%'
            OR lower(CommandLine) LIKE '%sharphound%'
            OR lower(CommandLine) LIKE '%bloodhound%'
            OR lower(CommandLine) LIKE '%impacket%'
            OR lower(CommandLine) LIKE '%seatbelt%'
            OR lower(ParentCommandLine) LIKE '%hacktool%'
            OR lower(ParentCommandLine) LIKE '%rubeus%'
            OR lower(ParentCommandLine) LIKE '%sharphound%'
            OR lower(ParentCommandLine) LIKE '%bloodhound%'
            OR lower(ParentCommandLine) LIKE '%impacket%'
            OR lower(ParentCommandLine) LIKE '%seatbelt%'
        )
        OR
        (
            (
                lower(CommandLine) LIKE '%reg%'
                AND lower(CommandLine) LIKE '%save%'
            )
            AND
            (
                lower(CommandLine) LIKE '%hklm%security%'
                OR lower(CommandLine) LIKE '%hklm%system%'
                OR lower(CommandLine) LIKE '%hklm%sam%'
            )
        )
    )
'''
)
huntCase.show(50, 70)

###   Possible webshell detection
##### description : Detects the execution of possible webshell

In [ ]:
huntCase = spark.sql(
'''
SELECT computer_name, ParentImage, ParentCommandLine, Image, CommandLine, OriginalFileName, CurrentDirectory, User
FROM sparktable
WHERE (source_name = "Microsoft-Windows-Sysmon" AND event_id = 1
        AND
        (
            lower(ParentImage) LIKE '%apache'
            OR lower(ParentImage) LIKE '%tomcat'
            OR lower(ParentImage) LIKE '%w3wp.exe' 
            OR lower(ParentImage) LIKE '%php-cgi.exe'
            OR lower(ParentImage) LIKE '%nginx.exe'
            OR lower(ParentImage) LIKE '%httpd.exe'
        )
        AND
        (
            lower(Description) LIKE '%forfiles - executes a command on selected files%'
            OR lower(Description) LIKE '%microsoft (r) html application host%'
            OR lower(Description) LIKE '%microsoft ® console based script host%'
            OR lower(Description) LIKE '%microsoft ® windows based script host%'
            OR lower(Description) LIKE '%microsoft(c) register server%'
            OR lower(Description) LIKE '%microsoft® html help executable%'
            OR lower(Description) LIKE '%msbuild.exe%'
            OR lower(Description) LIKE '%network command shell%'
            OR lower(Description) LIKE '%task scheduler configuration tool%'
            OR lower(Description) LIKE '%windows command processor%'
            OR lower(Description) LIKE '%windows host process (rundll32)%'
            OR lower(Description) LIKE '%windows powershell%'
            OR lower(Description) LIKE '%windows® installer%'
            OR lower(Description) LIKE '%wmi commandline utility%'
            OR lower(Description) LIKE '%wmi standard event consumer - scripting%'
        )
    )
'''
)
huntCase.show(10, 60)

###   Suspicious certutil execution detected
##### description : Detects the suspicious execution of certutil

In [ ]:
huntCase = spark.sql(
'''
SELECT computer_name, ParentImage, ParentCommandLine, Image, CommandLine, OriginalFileName, CurrentDirectory, User
FROM sparktable
WHERE (source_name = "Microsoft-Windows-Sysmon" AND event_id = 1
        AND
            lower(CommandLine) LIKE '%certutil.exe%'
        AND
        (
            lower(CommandLine) LIKE '%http%'
            OR lower(CommandLine) LIKE '%decode%'
            OR lower(CommandLine) LIKE '%url%'
            OR lower(CommandLine) LIKE '%ftp%'
            OR lower(CommandLine) LIKE '%ping%'
        )
    )
'''
)
huntCase.show(10, 60)

###   Suspicious execution via wscript.exe
##### description : Detects the suspicious usage of wscript

In [ ]:
huntCase = spark.sql(
'''
SELECT computer_name, ParentImage, ParentCommandLine, Image, CommandLine, OriginalFileName, User
FROM sparktable
WHERE (source_name = "Microsoft-Windows-Sysmon" AND event_id = 1
        AND
        (
            lower(CommandLine) LIKE '%wscript.exe%'
            OR lower(ParentCommandLine) LIKE '%wscript.exe%'
        )
        AND
        (
            lower(CommandLine) LIKE '%.jse%'
            OR lower(CommandLine) LIKE '%.vbe%'
            OR lower(CommandLine) LIKE '%.js%'
            OR lower(CommandLine) LIKE '%.vba%'
            OR lower(CommandLine) LIKE '%.vbs%'
        )
    )
'''
)
huntCase.show(10, 80)

###   Suspicious Regsvr32 CMD Executed
##### description : Detects the suspicious execution of Regsvr32 command

In [ ]:
huntCase = spark.sql(
'''
SELECT computer_name, ParentImage, ParentCommandLine, Image, CommandLine, OriginalFileName, User
FROM sparktable
WHERE (source_name = "Microsoft-Windows-Sysmon" AND event_id = 1
        AND
        (
            lower(OriginalFileName) LIKE '%regsvr32.exe%'
        )
        AND
        (
            lower(CommandLine) LIKE '%i:%'
        )
    )
'''
)
huntCase.show(10, 100)

###   Detects Suspicious svchost.exe CMD Execution
##### description : Detects the suspicious execution of svchost.exe command

In [ ]:
huntCase = spark.sql(
'''
SELECT computer_name, ParentImage, ParentCommandLine, Image, CommandLine, OriginalFileName, User
FROM sparktable
WHERE (source_name = "Microsoft-Windows-Sysmon" AND event_id = 1
        AND
            lower(Image) LIKE '%svchost.exe%'
        AND NOT
            (
                lower(ParentImage) LIKE '%services.exe%'
                OR lower(ParentImage) LIKE '%microsoft%windows defender%'
                OR lower(CommandLine) LIKE '%localservice%p%s%cdpsvc%'
            )
    )
'''
)
huntCase.show(10, 110)

###   Detects suspicious TASKKILL execution
##### description : Detects the suspicious usage of TASKKILL

In [ ]:
huntCase = spark.sql(
'''
SELECT computer_name, ParentImage, ParentCommandLine, Image, CommandLine, OriginalFileName, User
FROM sparktable
WHERE (source_name = "Microsoft-Windows-Sysmon" AND event_id = 1
        AND
            lower(Image) LIKE '%taskkill.exe%'
        AND
        (
            lower(ParentImage) LIKE '%cmd.exe%'
            OR lower(ParentImage) LIKE '%powershell.exe%'
        )
    )
'''
)
huntCase.show(10, 50)

###   Detects the potential Gootkit activity
##### description : Detects the potential Gootkit activity

In [ ]:
huntCase = spark.sql(
'''
SELECT computer_name, ParentImage, ParentCommandLine, Image, CommandLine, OriginalFileName, User
FROM sparktable
WHERE (source_name = "Microsoft-Windows-Sysmon" AND event_id = 1
        AND
            lower(CommandLine) LIKE '%collective_agreemente%'
        OR
        (
            lower(Description) LIKE '%windows powershell%'
            AND lower(ParentImage) LIKE '%\wscript.exe%'
        )

    )
'''
)
huntCase.show(10, 80)

###   WMI unusual child processes
##### description : Detects the unusual execution of child processes via WMI

In [ ]:
huntCase = spark.sql(
'''
SELECT computer_name, ParentImage, ParentCommandLine, Image, CommandLine, OriginalFileName, User
FROM sparktable
WHERE (source_name = "Microsoft-Windows-Sysmon" AND event_id = 1
        AND
            lower(ParentImage) LIKE '%\wmiprvse.exe%'
        AND
        (
            lower(Image) LIKE '%rundll32.exe%'
            OR lower(Image) LIKE '%regsvr32.exe%'
            OR lower(Image) LIKE '%mshta.exe%'
        )

    )
'''
)
huntCase.show(10, 80)

###   WMI Spawned From Unusual Process
##### description :  Detects the execution of WMI from unusual process

In [ ]:
huntCase = spark.sql(
'''
SELECT computer_name, ParentImage, ParentCommandLine, Image, CommandLine, OriginalFileName, User
FROM sparktable
WHERE (source_name = "Microsoft-Windows-Sysmon" AND event_id = 1
        AND
            lower(Image) LIKE '%\\wmiprvse.exe%'
        AND NOT
        (
            lower(ParentCommandLine) LIKE '%microsoft pfe remediation for configuration manager%pferemediationscript%'
            OR lower(ParentImage) LIKE '%svchost.exe%'
            OR lower(ParentImage) LIKE '%?%'
        )

    )
'''
)
huntCase.show(10, 80)

###   Potential WMI lateral movement detected
##### description :  Detects the ussage of WMI to perform Lateral Movement

In [ ]:
huntCase = spark.sql(
'''
SELECT computer_name, ParentImage, ParentCommandLine, Image, CommandLine, OriginalFileName, User
FROM sparktable
WHERE (source_name = "Microsoft-Windows-Sysmon" AND event_id = 1
        AND
        (
            lower(CommandLine) LIKE '%wmic%'
            AND lower(CommandLine) LIKE '%process%'
            AND lower(CommandLine) LIKE '%create%'
        )
        OR
        (
            lower(Description) == 'wmi commandline utility'
            AND lower(CommandLine) LIKE '%node%'
            AND
            (
                lower(CommandLine) LIKE '%process%'
                OR lower(CommandLine) LIKE '%service%'
                OR lower(CommandLine) LIKE '%shadowcopy%'
            )
        )

    )
'''
)
huntCase.show(50, 75)

###   HH.exe launched shell
##### description :  Detects a shell being launched by HH.exe

In [ ]:
huntCase = spark.sql(
'''
SELECT computer_name, ParentImage, ParentCommandLine, Image, CommandLine, OriginalFileName, CurrentDirectory, User
FROM sparktable
WHERE (source_name = "Microsoft-Windows-Sysmon" AND event_id = 1
        AND
        (
            lower(Description) LIKE '%microsoft® html help executable%'
        )

    )
'''
)
huntCase.show(100, 80)

###   Potential MSHTA.exe abuse
##### description :  Detects the potential MSHTA abuse

In [ ]:
huntCase = spark.sql(
'''
SELECT computer_name, ParentImage, ParentCommandLine, Image, CommandLine, OriginalFileName, CurrentDirectory, User
FROM sparktable
WHERE (source_name = "Microsoft-Windows-Sysmon" AND event_id = 1
        AND
        (
            lower(Description) LIKE '%microsoft (r) html application host%'
        )

    )
'''
)
huntCase.show(100, 85)

###  Potential MEGA/Rclone Data Exfiltration
##### description :  Detects Potential MEGA/Rclone Data Exfiltration

In [ ]:
huntCase = spark.sql(
'''
SELECT computer_name, ParentImage, ParentCommandLine, Image, CommandLine, OriginalFileName, CurrentDirectory, User
FROM sparktable
WHERE (source_name = "Microsoft-Windows-Sysmon" AND event_id = 1
        AND
        (
            lower(Image) LIKE '%rclone%'
            OR lower(Image) LIKE '%rsync.exe%'
            OR lower(Description) LIKE '%rclone%'
            OR lower(Description) LIKE '%rsync%'
        )
        AND
        (
            lower(CommandLine) LIKE '%config%'
            OR lower(CommandLine) LIKE '%lsd%'
            OR lower(CommandLine) LIKE '%remote%'
            OR lower(CommandLine) LIKE '%ftp:%'
            OR lower(CommandLine) LIKE '%--config%'
            OR lower(CommandLine) LIKE '%--auto-confirm%'
            OR lower(CommandLine) LIKE '%--milti-thread-streams%'
            OR lower(CommandLine) LIKE '%--progress%'
            OR lower(CommandLine) LIKE '%--no-check-certificate%'
            OR lower(CommandLine) LIKE '%--ignore-existing%'
            OR lower(CommandLine) LIKE '%--transfers%'
        )
    )
'''
)
huntCase.show(100, 80)

###  Lazarus Group Activity Detected
##### description : Detects the Lazarus group activity

In [ ]:
huntCase = spark.sql(
'''
SELECT computer_name, ParentImage, ParentCommandLine, Image, CommandLine, OriginalFileName, CurrentDirectory, User
FROM sparktable
WHERE (source_name = "Microsoft-Windows-Sysmon" AND event_id = 1
        AND
        (
            lower(Description) LIKE '%forfiles - executes a command on selected files%'
            OR lower(Description) LIKE '%microsoft (r) html application host%'
            OR lower(Description) LIKE '%microsoft ® console based script host%'
            OR lower(Description) LIKE '%microsoft ® windows based script host%'
            OR lower(Description) LIKE '%microsoft(c) register server%'
            OR lower(Description) LIKE '%microsoft® html help executable%'
            OR lower(Description) LIKE '%msbuild.exe%'
            OR lower(Description) LIKE '%network command shell%'
            OR lower(Description) LIKE '%task scheduler configuration tool%'
            OR lower(Description) LIKE '%windows command processor%'
            OR lower(Description) LIKE '%windows host process (rundll32)%'
            OR lower(Description) LIKE '%windows powershell%'
            OR lower(Description) LIKE '%windows® installer%'
            OR lower(Description) LIKE '%wmi commandline utility%'
            OR lower(Description) LIKE '%wmi standard event consumer - scripting%'
        )
        AND
        (
            lower(CommandLine) LIKE '%?:%drivers%'
        )
    )
'''
)
huntCase.show(10, 80)

###  Detects suspicious MSBuild execution
##### description : Detects the suspicious execution of MSBuild

In [ ]:
huntCase = spark.sql(
'''
SELECT computer_name, ParentImage, ParentCommandLine, Image, CommandLine, OriginalFileName, CurrentDirectory, User
FROM sparktable
WHERE (source_name = "Microsoft-Windows-Sysmon" AND event_id = 1
        AND
        (
            lower(Image) LIKE '%msbuild.exe%'
        )
        OR
        (
            lower(CommandLine) LIKE '%msbuild %'
        )
    )
'''
)
huntCase.show(100, 80)

###  Masqueraded Windows/Sysinternal Binary Detected
##### description : Detects Windows/Sysinternal Binary Masquerading attack

In [ ]:
huntCase = spark.sql(
'''
SELECT computer_name, ParentImage, ParentCommandLine, Image, CommandLine, Description, CurrentDirectory, User
FROM sparktable
WHERE (source_name = "Microsoft-Windows-Sysmon" AND event_id = 1
        AND
        (
            lower(Description) LIKE '%execute processes remotely%'
            OR lower(Description) LIKE '%windows command processor%'
            OR lower(Description) LIKE '%whoami - displays logged on user information%' 
            OR lower(Description) LIKE '%registry editor%'
            OR lower(Description) LIKE '%secure file delete%'
            OR lower(Description) LIKE '%net command%'
            OR lower(Description) LIKE '%microsoft® windows based script host%'
            OR lower(Description) LIKE '%autostart program viewer%'
            OR lower(Description) LIKE '%microsoft (r) html application host%'
            OR lower(Description) LIKE '%microsoft® html help executable%'
            OR lower(Description) LIKE '%remote desktop connection%'
            OR lower(Description) LIKE '%network command shell%'
            OR lower(Description) LIKE '%registry console tool%'
            OR lower(Description) LIKE '%run as utility%'
            OR lower(Description) LIKE '%task scheduler configuration tool%'
            OR lower(Description) LIKE '%services and controller app%'
            OR lower(Description) LIKE '%service control manager configuration tool%'
            OR lower(Description) LIKE '%command line interface for microsoft® volume shadow copy service%'
            OR lower(Description) LIKE '%microsoft(c) register server%'
            OR lower(Description) LIKE '%reports effective permissions for securable objects%'
            OR lower(Description) LIKE '%accessenum%'
            OR lower(Description) LIKE '%active directory editor%'
            OR lower(Description) LIKE '%active directory ldap monitor%'
            OR lower(Description) LIKE '%autologon configuration%'
            OR lower(Description) LIKE '%handle viewer%'
            OR lower(Description) LIKE '%creates pending movefile operations%'
            OR lower(Description) LIKE '%driver bug test program%'
            OR lower(Description) LIKE '%sysinternals process dump utility%'
            OR lower(Description) LIKE '%lists files and directories opened remotely%'
            OR lower(Description) LIKE '%local and remote system information viewer%'
            OR lower(Description) LIKE '%terminates processes on local or remote systems%'
            OR lower(Description) LIKE '%local and remote password changer%'
            OR lower(Description) LIKE '%service information and configuration utility%'
            OR lower(Description) LIKE '%process suspender%'
            OR lower(Description) LIKE '%run as different user%'
            OR lower(Description) LIKE '%search for ansi and unicode strings in binary images%'
            OR lower(Description) LIKE '%sysinternals winobj or schedule service command line interface%'
            OR lower(Description) LIKE '%bits administration utility%'
            OR lower(Description) LIKE '%certutil.exe%'
            OR lower(Description) LIKE '%microsoft ® console based script host%'
            OR lower(Description) LIKE '%com surrogate%'
            OR lower(Description) LIKE '%extensible storage engine utilities for microsoft(r) windows(r)%'
            OR lower(Description) LIKE '%windows host process (rundll32)%'
            OR lower(Description) LIKE '%visual basic command line compiler%'
            OR lower(Description) LIKE '%terminates processes%'
            OR lower(Description) LIKE '%ctf loader%'
            OR lower(Description) LIKE '%file transfer program%'
            OR lower(Description) LIKE '%system configuration utility%'
            OR lower(Description) LIKE '%windows remote assistance%'
            OR lower(Description) LIKE '%microsoft® logon server test utility%'
            OR lower(Description) LIKE '%displays the current open files list%'
            OR lower(Description) LIKE '%mimikatz for windows%'
            OR lower(Description) LIKE '%microsoft application virtualization injector%'
            OR lower(Description) LIKE '%mavinject32%'
        )
        AND NOT
        (
            lower(Image) LIKE '%psexec.exe'
            OR lower(Image) LIKE '%psexec.exe'
            OR lower(Image) LIKE '%psexec64.exe'
            OR lower(Image) LIKE '%remoteexec.exe'
            OR lower(Image) LIKE '%cmd.exe'
            OR lower(Image) LIKE '%whoami.exe'
            OR lower(Image) LIKE '%regedit.exe'
            OR lower(Image) LIKE '%sdelete.exe'
            OR lower(Image) LIKE '%sdelete64.exe'
            OR lower(Image) LIKE '%net.exe'
            OR lower(Image) LIKE '%net1.exe'
            OR lower(Image) LIKE '%wscript.exe'
            OR lower(Image) LIKE '%autoruns.exe'
            OR lower(Image) LIKE '%autoruns64.exe'
            OR lower(Image) LIKE '%autorunsc.exe'
            OR lower(Image) LIKE '%autorunsc64.exe'
            OR lower(Image) LIKE '%mshta.exe'
            OR lower(Image) LIKE '%hh.exe'
            OR lower(Image) LIKE '%adfind.exe'
            OR lower(Image) LIKE '%mstsc.exe'
            OR lower(Image) LIKE '%multidesk.exe'
            OR lower(Image) LIKE '%netsh.exe'
            OR lower(Image) LIKE '%reg.exe'
            OR lower(Image) LIKE '%reg_mam.exe'
            OR lower(Image) LIKE '%runas.exe'
            OR lower(Image) LIKE '%schtasks.exe'
            OR lower(Image) LIKE '%services.exe'
            OR lower(Image) LIKE '%sc.exe'
            OR lower(Image) LIKE '%vssadmin.exe'
            OR lower(Image) LIKE '%regsvr32.exe'
            OR lower(Image) LIKE '%accesschk.exe'
            OR lower(Image) LIKE '%accesschk64.exe'
            OR lower(Image) LIKE '%accessenum.exe'
            OR lower(Image) LIKE '%adexplorer.exe'
            OR lower(Image) LIKE '%adexplorer64.exe'
            OR lower(Image) LIKE '%adinsight.exe'
            OR lower(Image) LIKE '%adinsight64.exe'
            OR lower(Image) LIKE '%adinsight64.exe'
            OR lower(Image) LIKE '%autologon.exe'
            OR lower(Image) LIKE '%autologon64.exe'
            OR lower(Image) LIKE '%handle.exe'
            OR lower(Image) LIKE '%handle64.exe'
            OR lower(Image) LIKE '%movefile.exe'
            OR lower(Image) LIKE '%movefile64.exe'
            OR lower(Image) LIKE '%notmyfault.exe'
            OR lower(Image) LIKE '%notmyfault64.exe'
            OR lower(Image) LIKE '%notmyfaultc.exe'
            OR lower(Image) LIKE '%notmyfaultc64.exe'
            OR lower(Image) LIKE '%procdump.exe'
            OR lower(Image) LIKE '%procdump64.exe'
            OR lower(Image) LIKE '%psfile.exe'
            OR lower(Image) LIKE '%psfile64.exe'
            OR lower(Image) LIKE '%psinfo.exe'
            OR lower(Image) LIKE '%psinfo64.exe'
            OR lower(Image) LIKE '%pskill.exe'
            OR lower(Image) LIKE '%pskill64.exe'
            OR lower(Image) LIKE '%pskllsvc.exe'
            OR lower(Image) LIKE '%pspasswd.exe'
            OR lower(Image) LIKE '%pspasswd64.exe'
            OR lower(Image) LIKE '%psservice.exe'
            OR lower(Image) LIKE '%psservice64.exe'
            OR lower(Image) LIKE '%pssuspend.exe'
            OR lower(Image) LIKE '%pssuspend64.exe'
            OR lower(Image) LIKE '%shellrunas.exe'
            OR lower(Image) LIKE '%strings.exe'
            OR lower(Image) LIKE '%strings64.exe'
            OR lower(Image) LIKE '%winobj.exe'
            OR lower(Image) LIKE '%at.exe'
            OR lower(Image) LIKE '%bitsadmin.exe'
            OR lower(Image) LIKE '%certutil.exe'
            OR lower(Image) LIKE '%cscript.exe'
            OR lower(Image) LIKE '%dllhost.exe'
            OR lower(Image) LIKE '%esentutl.exe'
            OR lower(Image) LIKE '%rundll32.exe'
            OR lower(Image) LIKE '%rundll32-low.exe'
            OR lower(Image) LIKE '%vbc.exe'
            OR lower(Image) LIKE '%taskkill.exe'
            OR lower(Image) LIKE '%ctfmon.exe'
            OR lower(Image) LIKE '%ftp.exe'
            OR lower(Image) LIKE '%msconfig.exe'
            OR lower(Image) LIKE '%msra.exe'
            OR lower(Image) LIKE '%nltest.exe'
            OR lower(Image) LIKE '%openfiles.exe'
            OR lower(Image) LIKE '%mimikatz.exe'
            OR lower(Image) LIKE '%mavinject.exe'
            OR lower(Image) LIKE '%mavinject64.exe'
            OR lower(Image) LIKE '%mavinject32.exe'
            OR lower(Image) LIKE '%mavinject32.exe'
        )
        AND NOT
        (
             lower(ParentCommandLine) LIKE '%system32%svchost.exe -k netsvcs -p -s%'
        )
    )
'''
)
huntCase.show(10, 60)

### Suspicious usage of CScript detected
##### description :  Detects the suspicious usage of CScript

In [ ]:
huntCase = spark.sql(
'''
SELECT computer_name, ParentImage, ParentCommandLine, Image, CommandLine, OriginalFileName, CurrentDirectory, User
FROM sparktable
WHERE (source_name = "Microsoft-Windows-Sysmon" AND event_id = 1
        AND
        (
            lower(OriginalFileName) LIKE '%cscript.exe%'
        )
        AND
        (
            lower(CommandLine) LIKE '%.jse%'
            OR lower(CommandLine) LIKE '%.vbe%'
            OR lower(CommandLine) LIKE '%.js%'
            OR lower(CommandLine) LIKE '%.vba%'
            OR lower(CommandLine) LIKE '%.vbs%'
        )
    )
'''
)
huntCase.show(100, 80)

### Volume Shadow copy deletion detected
##### description :  Detects volume shadow copy deletion

In [ ]:
huntCase = spark.sql(
'''
SELECT computer_name, ParentImage, ParentCommandLine, Image, CommandLine, OriginalFileName, CurrentDirectory, User
FROM sparktable
WHERE (source_name = "Microsoft-Windows-Sysmon" AND event_id = 1
        AND
        (
            lower(CommandLine) LIKE '%vssadmin%'
        )
        AND
        (
            lower(CommandLine) LIKE '%delete%' 
            AND lower(CommandLine) LIKE '%shadow%'
        )
    )
'''
)
huntCase.show(10, 80)

### Suspicious File Permission Modification - Granting Access to Network Shares via Net Share
##### description :  Detects attempts by attackers to modify Windows Access Control Lists (ACLS) via the net share

In [ ]:
huntCase = spark.sql(
'''
SELECT computer_name, ParentImage, ParentCommandLine, Image, CommandLine, OriginalFileName, CurrentDirectory, User
FROM sparktable
WHERE (source_name = "Microsoft-Windows-Sysmon" AND event_id = 1
        AND
        (
            lower(Description) LIKE '%net command%'
        )
        AND
        (
            lower(CommandLine) LIKE '%share%' 
            AND lower(CommandLine) LIKE '%grant%'
        )
    )
'''
)
huntCase.show(10, 80)

### Potential Runexehelper Proxy Execution Detected
##### description : Detects Potential Runexehelper Proxy Execution

In [ ]:
huntCase = spark.sql(
'''
SELECT computer_name, ParentImage, ParentCommandLine, Image, CommandLine, OriginalFileName, CurrentDirectory, User
FROM sparktable
WHERE (source_name = "Microsoft-Windows-Sysmon" AND event_id = 1
        AND
        (
            lower(Image) LIKE '%runexehelper.exe%'
        )
    )
'''
)
huntCase.show(10, 80)

### AppInstaller Installation Detected
##### description : Detects the installation of AppInstaller

In [ ]:
huntCase = spark.sql(
'''
SELECT computer_name, ParentImage, ParentCommandLine, Image, CommandLine, OriginalFileName, CurrentDirectory, User
FROM sparktable
WHERE (source_name = "Microsoft-Windows-Sysmon" AND event_id = 1
        AND
        (
            lower(Image) LIKE '%appinstaller.exe%'
        )
    )
'''
)
huntCase.show(10, 80)

### Impacket Wmiexec Detected
##### description: Detects Impacket Wmiexec

In [ ]:
huntCase = spark.sql(
'''
SELECT computer_name, ParentImage, ParentCommandLine, Image, CommandLine, OriginalFileName, CurrentDirectory, User
FROM sparktable
WHERE (source_name = "Microsoft-Windows-Sysmon" AND event_id = 1
        AND
        (
            lower(CommandLine) LIKE '%127.0.0.1%admin$%'
            OR lower(ParentCommandLine) LIKE '%127.0.0.1%admin$%'
        )
        AND
        (
            (
                lower(CommandLine) LIKE '%cmd.exe%'
                AND lower(CommandLine) LIKE '%/q%'
                AND lower(CommandLine) LIKE '%/c%'
            )
            OR
            (
                lower(ParentCommandLine) LIKE '%cmd.exe%'
                AND lower(ParentCommandLine) LIKE '%/q%'
                AND lower(ParentCommandLine) LIKE '%/c%'
            )
        )
    )
'''
)
huntCase.show(10, 80)

### CobaltStrike Werfault Detected
##### description: Detects CobaltStrike injected into Werfault

In [ ]:
huntCase = spark.sql(
'''
SELECT computer_name, ParentImage, ParentCommandLine, Image, CommandLine, OriginalFileName, CurrentDirectory, User
FROM sparktable
WHERE (source_name = "Microsoft-Windows-Sysmon" AND event_id = 1
        AND
        (
            lower(CommandLine) LIKE '%werfault.exe%'
        )
        AND
        (
            (
                lower(CommandLine) LIKE '%-s%-t%'
                AND lower(CommandLine) LIKE '%-o 0%'
            )
            OR
            (
                lower(ParentCommandLine) LIKE '%-s%-t%'
                AND lower(ParentCommandLine) LIKE '%-o 0%'
            )
        )
    )
'''
)
huntCase.show(10, 60)

### Suspicious non EXE image Detected
##### description: Detects suspicious non exe image execution

In [ ]:
huntCase = spark.sql(
'''
SELECT computer_name, ParentImage, ParentCommandLine, Image, CommandLine, OriginalFileName, CurrentDirectory, User
FROM sparktable
WHERE (source_name = "Microsoft-Windows-Sysmon" AND event_id = 1
        AND NOT
        (
            lower(Image) LIKE '%.scr%'
            OR lower(Image) LIKE '%.dat%'
            OR lower(Image) LIKE '%\chcp.com%'
            OR lower(Image) LIKE '%.p5x%'
            OR lower(Image) LIKE '%.tmp%'
            OR lower(Image) LIKE '%\mode.com%'
            OR lower(Image) LIKE '%\sftmime.com%'
            OR lower(Image) LIKE '%\cgi%'
            OR lower(Image) LIKE '%.bin%'
            OR lower(Image) LIKE '%-\winscp.com%'
            OR lower(Image) LIKE '%\more.com%'
            OR lower(Image) LIKE '%.srv%'
            OR lower(Image) LIKE '%.exe%'
            OR lower(Image) LIKE '%basename%'
            OR lower(Image) LIKE '%c%windows%temp%'
            OR lower(Image) LIKE '%trend micro%'
            OR lower(Image) LIKE '%hpimageassistant%'
            OR lower(Image) LIKE '%wd desktop app%'
        )
    )
'''
)
huntCase.show(10, 60)

### Detects MMC20.Application Lateral Movement
##### description: Detects MMC20.Application Lateral Movement

### Detects MMC20.Application Lateral Movement
##### description: Detects MMC20.Application Lateral Movement

In [ ]:
huntCase = spark.sql(
'''
SELECT computer_name, ParentImage, ParentCommandLine, Image, CommandLine, OriginalFileName, CurrentDirectory, User
FROM sparktable
WHERE (source_name = "Microsoft-Windows-Sysmon" AND event_id = 1
        AND
        (
            lower(CommandLine) LIKE '%-embedding%'
        )
        AND
        (
            lower(ParentImage) LIKE '%svchost.exe%'
        )
        AND
        (
            lower(Image) LIKE '%mmc.exe%'
        )
        AND NOT
        (
            lower(ParentCommandLine) LIKE '%svchost.exe -k dcomlaunch%'
        )
    )
'''
)
huntCase.show(10, 80)

### Execution from Remote Shares Lateral Movement
##### description: Detects the execution from remote shares for Lateral Movement

In [ ]:
huntCase = spark.sql(
'''
SELECT computer_name, ParentImage, ParentCommandLine, Image, CommandLine, OriginalFileName, CurrentDirectory, User
FROM sparktable
WHERE (source_name = "Microsoft-Windows-Sysmon" AND event_id = 1
        AND
        (
            lower(Image) LIKE '%admin$%'
            OR lower(Image) LIKE '%ipc$%'
            OR lower(Image) LIKE '%c$%'
            OR lower(Image) LIKE '%\public%'
        )
    )
'''
)
huntCase.show(10, 80)

### Detects execution via PSRemoting - Lateral Movement
##### description: Detects the lateral movement via PSRemoting

In [ ]:
huntCase = spark.sql(
'''
SELECT computer_name, ParentImage, ParentCommandLine, Image, CommandLine, OriginalFileName, CurrentDirectory, User
FROM sparktable
WHERE (source_name = "Microsoft-Windows-Sysmon" AND event_id = 1
        AND
        (
            lower(ParentCommandLine) LIKE '%-embedding%'
            OR lower(ParentCommandLine) LIKE '%-enb%'
        )
        AND
        (
            lower(ParentImage) LIKE '%wsmprovhost.exe%'
        )
    )
'''
)
huntCase.show(10, 80)

### WMI Lateral Movement detected
##### description: Detects the lateral movement via WMI execution

In [ ]:
huntCase = spark.sql(
'''
SELECT computer_name, ParentImage, ParentCommandLine, Image, CommandLine, OriginalFileName, CurrentDirectory, User
FROM sparktable
WHERE (source_name = "Microsoft-Windows-Sysmon" AND event_id = 1
        AND
        (
            lower(ParentCommandLine) LIKE '%-embedding%'
        )
        AND
        (
            lower(ParentImage) LIKE '%\wmiprvse.exe%'
        )
        AND NOT
        (
            lower(Image) LIKE '%-%-%-%-%'
        )
    )
'''
)
huntCase.show(10, 100)

### WFC Proxy Execution
##### description:  Detects the execution of WFC to compile and execute C# code embedded in XOML file

In [ ]:
huntCase = spark.sql(
'''
SELECT computer_name, ParentImage, ParentCommandLine, Image, CommandLine, OriginalFileName, CurrentDirectory, User
FROM sparktable
WHERE (source_name = "Microsoft-Windows-Sysmon" AND event_id = 1
        AND
        (
            lower(Description) == 'microsoft windows workflow foundation compiler'
        )
    )
'''
)
huntCase.show(10, 80)

### Microsoft Workflow Compiler Execution detected
##### description:  Detects the execution of Microsoft Workflow Compiler

In [ ]:
huntCase = spark.sql(
'''
SELECT computer_name, ParentImage, ParentCommandLine, Image, CommandLine, Description, CurrentDirectory, User
FROM sparktable
WHERE (source_name = "Microsoft-Windows-Sysmon" AND event_id = 1
        AND
        (
            lower(Image) LIKE '%compiler.exe%'
        )
        OR
        (
            lower(Description) == 'microsoft.workflow.compiler.exe'
        )
        AND NOT lower(CommandLine) == 'null'
    )
'''
)
huntCase.show(10, 60)

### Suspicious JSC Execution detected
##### description: Detects suspicious execution of JSC

In [ ]:
huntCase = spark.sql(
'''
SELECT computer_name, ParentImage, ParentCommandLine, Image, CommandLine, OriginalFileName, CurrentDirectory, User
FROM sparktable
WHERE (source_name = "Microsoft-Windows-Sysmon" AND event_id = 1
        AND
        (
            lower(Image) LIKE '%jsc.exe%'
        )
    )
'''
)
huntCase.show(10, 80)

### Visual Basics Commandline Compiler Proxy Execution
##### description: Detects the execution of VBC to compile and execute the code

In [ ]:
huntCase = spark.sql(
'''
SELECT computer_name, ParentImage, ParentCommandLine, Image, CommandLine, OriginalFileName, CurrentDirectory, User
FROM sparktable
WHERE (source_name = "Microsoft-Windows-Sysmon" AND event_id = 1
        AND
        (
            lower(Description) == 'visual basic command line compiler'
        )
    )
'''
)
huntCase.show(10, 80)

### System Service Modification Using SC/PowerShell
##### description: Detects the System Service Modification Using SC/PowerShell

In [ ]:
huntCase = spark.sql(
'''
SELECT computer_name, ParentImage, ParentCommandLine, Image, CommandLine, User
FROM sparktable
WHERE (source_name = "Microsoft-Windows-Sysmon" AND event_id = 1
        AND
        (
            (
                lower(CommandLine) LIKE '%sc.exe%'
                AND lower(CommandLine) LIKE '%config%'
            )
            AND
            (
                lower(CommandLine) LIKE '%disabled%'
                OR lower(CommandLine) LIKE '%auto%'
            )
            OR
            (
                lower(CommandLine) LIKE '%set-service%'
                OR lower(CommandLine) LIKE '%startuptype%'
                OR lower(CommandLine) LIKE '%new-service%'
            )
            OR
            (
                lower(ParentCommandLine) LIKE '%set-service%'
                OR lower(ParentCommandLine) LIKE '%startuptype%'
                OR lower(ParentCommandLine) LIKE '%new-service%'
            )
        )
    )
'''
)
huntCase.show(10, 80)

### SSH Protocol Tunneling detected
##### description: Detects the protocol tunneling

In [ ]:
huntCase = spark.sql(
'''
SELECT computer_name, ParentImage, ParentCommandLine, Image, CommandLine, OriginalFileName, CurrentDirectory, User
FROM sparktable
WHERE (source_name = "Microsoft-Windows-Sysmon" AND event_id = 1
        AND
        (
            lower(Description) LIKE '%rlogin%'
        )
        AND
        (
            lower(CommandLine) LIKE '%pw'
            AND lower(CommandLine) LIKE '%-r'
        )
    )
'''
)
huntCase.show(10, 80)

### File Download via IMEWDBLD.exe
##### description: Detects attempts to conduct file downloads using the Microsoft IME Open Extended Dictionary Module (IMEWDBLD.exe) lolbin.

In [ ]:
huntCase = spark.sql(
'''
SELECT computer_name, ParentImage, ParentCommandLine, Image, CommandLine, OriginalFileName, CurrentDirectory, User
FROM sparktable
WHERE (source_name = "Microsoft-Windows-Sysmon" AND event_id = 1
        AND
        (
            lower(Description) == 'microsoft ime open extended dictionary module'
            OR lower(Image) LIKE '%imewdbld.exe%'
        )
        AND
        (
            lower(CommandLine) LIKE '%http%'
        )
    )
'''
)
huntCase.show(10, 80)

### File Download via Finger.exe
##### description: Detects attempts to use the Finger.exe tool for potential ingress tool transfer

In [ ]:
huntCase = spark.sql(
'''
SELECT computer_name, ParentImage, ParentCommandLine, Image, CommandLine, OriginalFileName, CurrentDirectory, User
FROM sparktable
WHERE (source_name = "Microsoft-Windows-Sysmon" AND event_id = 1
        AND
        (
            lower(Description) == 'tcpip finger command'
        )
    )
'''
)
huntCase.show(10, 80)

### Credential Dumping via comsvcs.dll
##### description: Detects attempts to dump process memory using the built in Windows comsvcs.dll

In [ ]:
huntCase = spark.sql(
'''
SELECT computer_name, ParentImage, ParentCommandLine, Image, CommandLine, OriginalFileName, CurrentDirectory, User
FROM sparktable
WHERE (source_name = "Microsoft-Windows-Sysmon" AND event_id = 1
        AND
        (
            lower(CommandLine) LIKE '%minidump%'
            AND lower(CommandLine) LIKE '%comsvcs.dll%'
        )
    )
'''
)
huntCase.show(10, 50)

### Esentutl.exe Abuse Detected
##### description: Detects suspicious Esentutl.exe usage

In [ ]:
huntCase = spark.sql(
'''
SELECT computer_name, ParentImage, ParentCommandLine, Image, CommandLine, OriginalFileName, CurrentDirectory, User
FROM sparktable
WHERE (source_name = "Microsoft-Windows-Sysmon" AND event_id = 1
        AND
        (
            lower(Image) LIKE '%\esentutl.exe%'
            OR lower(Description) == 'extensible storage engine utilities for microsoft(r) windows(r)'
        )
        AND NOT
        (
            lower(CommandLine) LIKE '%/k %'
        )
    )
'''
)
huntCase.show(10, 70)

### WMIC Shadow Copy Abuse Detected
##### description: Detects the use of the WMI commandline utility to manipulate shadow copies, including remotely

In [ ]:
huntCase = spark.sql(
'''
SELECT computer_name, ParentImage, ParentCommandLine, Image, CommandLine, Description, User
FROM sparktable
WHERE (source_name = "Microsoft-Windows-Sysmon" AND event_id = 1
        AND
        (
            lower(CommandLine) LIKE '%shadowcopy%'
            OR lower(Description) == 'wmi commandline utility'
        )
        AND NOT
            lower(ParentCommandLine) LIKE '%svchost.exe%-k%netsvcs%'
    )
'''
)
huntCase.show(20, 40)

### NTDSUtil Active Directory Database Dumping Detected
##### description:  Detects the use of the Active Directory management tool ntdsutil.exe to dump a copy of the Active Directory Database

In [ ]:
huntCase = spark.sql(
'''
SELECT computer_name, ParentImage, ParentCommandLine, Image, CommandLine, OriginalFileName, CurrentDirectory, User
FROM sparktable
WHERE (source_name = "Microsoft-Windows-Sysmon" AND event_id = 1
        AND
        (
            lower(CommandLine) LIKE '%ntdsutil%'
            AND lower(CommandLine) LIKE '%ntds%'
        )
    )
'''
)
huntCase.show(10, 70)

### IFEO Debugger Abuse Detected
##### description: Detects attempts to escalate privilege/establish persistence by executing malicious content triggered by Image File Execution Options (IFEO) debuggers

In [ ]:
huntCase = spark.sql(
'''
SELECT computer_name, ParentImage, ParentCommandLine, Image, CommandLine, OriginalFileName, CurrentDirectory, User
FROM sparktable
WHERE (source_name = "Microsoft-Windows-Sysmon" AND event_id = 1
        AND
        (
            lower(Description) == 'microsoft® nt global flags manipulator'
        )
    )
'''
)
huntCase.show(10, 70)

### Netsh Helper DLL Registration Detected
##### description: Detects attempts to establish persistence or escalate privilege by triggering Netshell helper DLLs

In [ ]:
huntCase = spark.sql(
'''
SELECT computer_name, ParentImage, ParentCommandLine, Image, CommandLine, OriginalFileName, CurrentDirectory, User
FROM sparktable
WHERE (source_name = "Microsoft-Windows-Sysmon" AND event_id = 1
        AND
        (
            lower(CommandLine) LIKE '%netsh%'
            AND lower(CommandLine) LIKE '%add%'
            AND lower(CommandLine) LIKE '%helper%'
        )
    )
'''
)
huntCase.show(10, 70)

### Csc.exe Compilation After Delivery Detected
##### description: Detects attempts to use the csc.exe .NET C# compiler to compile payloads from downloaded C# code

In [ ]:
huntCase = spark.sql(
'''
SELECT computer_name, ParentImage, ParentCommandLine, Image, CommandLine, CurrentDirectory, User
FROM sparktable
WHERE (source_name = "Microsoft-Windows-Sysmon" AND event_id = 1
        AND
        (
            lower(Description) == 'visual c# command line compiler'
        )
    )
'''
)
huntCase.show(10, 70)

### Signed Binary Proxy Execution via Odbcconf Detected
##### description: Detects abuse of the Microsoft signed odbcconf.exe (Open Database Connectivity Configuration) binary to proxy the execution of malicious payloads

In [ ]:
huntCase = spark.sql(
'''
SELECT computer_name, ParentImage, ParentCommandLine, Image, CommandLine, OriginalFileName, CurrentDirectory, User
FROM sparktable
WHERE (source_name = "Microsoft-Windows-Sysmon" AND event_id = 1
        AND
        (
            lower(Description) == 'odbc driver configuration program'
        )
    )
'''
)
huntCase.show(10, 70)

### Signed Binary Proxy Execution via Mavinject Detected
##### description: Detects abuse of the Microsoft Application Virtualization Injector (Mavinject.exe) to inject malicious DLLs into running processes

In [ ]:
huntCase = spark.sql(
'''
SELECT computer_name, ParentImage, ParentCommandLine, Image, CommandLine, OriginalFileName, CurrentDirectory, User
FROM sparktable
WHERE (source_name = "Microsoft-Windows-Sysmon" AND event_id = 1
        AND
        (
            lower(Description) == 'microsoft application virtualization injector'
            AND lower(CommandLine) LIKE '%.dll%'
        )
    )
'''
)
huntCase.show(10, 50)

### MSXSL Script Bypass Detected
##### description: Detects attempts to execute scripts embedded in Extensible Stylesheet Language (XSL) files via the Microsoft Common Line Transformation Utility (msxsl.exe)

In [ ]:
huntCase = spark.sql(
'''
SELECT computer_name, ParentImage, ParentCommandLine, Image, CommandLine, OriginalFileName, CurrentDirectory, User
FROM sparktable
WHERE (source_name = "Microsoft-Windows-Sysmon" AND event_id = 1
        AND
        (
            lower(Description) == 'msxsl'
        )
    )
'''
)
huntCase.show(10, 70)

### WMIC XSL Script Bypass Detected
##### description: Detects attempts to execute scripts embedded in Extensible Stylesheet Language (XSL) files via the Windows Management Instrumentation Command-Line utility (wmic.exe)

In [ ]:
huntCase = spark.sql(
'''
SELECT computer_name, ParentImage, ParentCommandLine, Image, CommandLine, OriginalFileName, CurrentDirectory, User
FROM sparktable
WHERE (source_name = "Microsoft-Windows-Sysmon" AND event_id = 1
        AND
        (
            lower(CommandLine) LIKE '%format%'
            AND lower(CommandLine) LIKE '%xsl%'
        )
        AND NOT
        (
            lower(CommandLine) LIKE '%list%'
            OR lower(CommandLine) LIKE '%csv%'
            OR lower(CommandLine) LIKE '%namespace%'
            OR lower(CommandLine) LIKE '%value%'
        )
    )
'''
)
huntCase.show(10, 70)

### Suspicious File Permission Modification - Granting Access to Files/Folders
##### description: Detects attempts by attackers to modify Windows Access Control Lists (ACLS) via the Windows icacls.exe tool in order to grant access to files and folders

In [ ]:
huntCase = spark.sql(
'''
SELECT computer_name, ParentImage, ParentCommandLine, Image, CommandLine, User
FROM sparktable
WHERE (source_name = "Microsoft-Windows-Sysmon" AND event_id = 1
        AND
        (
            lower(OriginalFileName) LIKE '%icacls.exe%'
        )
        AND
        (
            lower(CommandLine) LIKE '%grant%'
            OR
            (
                lower(CommandLine) LIKE '%administrators%'
                AND lower(CommandLine) LIKE '%everyone%'
            )
        )
    )
'''
)
huntCase.show(10, 70)

### CMSTP Proxy Execution Detected
##### description: Detects attempts to abuse the Microsoft Connection Manager Profile Installer (CMSTP.exe) to proxy execution of malicious code

In [ ]:
huntCase = spark.sql(
'''
SELECT computer_name, ParentImage, ParentCommandLine, Image, CommandLine, CurrentDirectory, User
FROM sparktable
WHERE (source_name = "Microsoft-Windows-Sysmon" AND event_id = 1
        AND
        (
            lower(Description) LIKE 'microsoft connection manager profile installer'
        )
        AND
        (
            lower(CommandLine) LIKE '%/au%'
            OR lower(CommandLine) LIKE '%/s%'
            OR lower(CommandLine) LIKE '%/su$'

        )
    )
'''
)
huntCase.show(10, 70)

###  Msiexec Proxy Execution Detected
##### description: Detects attempts to abuse the Windows Installer (msiexec.exe) command-line utility to proxy the execution of malicious DLL payloads

In [ ]:
huntCase = spark.sql(
'''
SELECT computer_name, ParentImage, ParentCommandLine, Image, CommandLine, User
FROM sparktable
WHERE (source_name = "Microsoft-Windows-Sysmon" AND event_id = 1
        AND
        (
            lower(Description) == 'windows® installer'
        )
        AND
        (
            lower(CommandLine) LIKE '%/quiet%'
            OR lower(CommandLine) LIKE '%/q%'
            OR lower(CommandLine) LIKE '%/y$'

        )
        AND
        (
            lower(ParentImage) LIKE '%\powershell.exe%'
            OR lower(ParentImage) LIKE '%\pwsh.exe%'
            OR lower(ParentImage) LIKE '%\cmd.exe%'
            OR lower(ParentImage) LIKE '%\wt.exe%'
            OR lower(ParentImage) LIKE '%\winword.exe%'
            OR lower(ParentImage) LIKE '%\excel.exe%'
            OR lower(ParentImage) LIKE '%\powerpnt.exe%'
            OR lower(ParentImage) LIKE '%\mspub.exe%'
            OR lower(ParentImage) LIKE '%\visio.exe%'
            OR lower(ParentImage) LIKE '%\msaccess.exe%'
            OR lower(ParentImage) LIKE '%\eqnedt32.exe%'
            OR lower(ParentImage) LIKE '%\explorer.exe%'
        )
    )
'''
)
huntCase.show(10, 100)

###  Regasm Unregister Proxy Execution Detected
##### description: Detects attempts to abuse the Microsoft Regasm .NET COM registration command-line utility to proxy execute malicious code using the binary's UnRegisterClass function

In [ ]:
huntCase = spark.sql(
'''
SELECT computer_name, ParentImage, ParentCommandLine, Image, CommandLine, User
FROM sparktable
WHERE (source_name = "Microsoft-Windows-Sysmon" AND event_id = 1
        AND
        (
            lower(Image) LIKE '%regasm.exe%'
        )
        AND
        (
            lower(CommandLine) LIKE '%/u%'

        )
    )
'''
)
huntCase.show(10, 70)

###  Regsvcs Unregister Proxy Execution Detected
##### description: Detects attempts to abuse the Microsoft Regsvr .NET COM registration command-line utility to proxy execute malicious code using the binary's UnRegisterClass function

In [ ]:
huntCase = spark.sql(
'''
SELECT computer_name, ParentImage, ParentCommandLine, Image, CommandLine, User
FROM sparktable
WHERE (source_name = "Microsoft-Windows-Sysmon" AND event_id = 1
        AND
        (
            lower(Description) == 'microsoft .net services installation utility'
        )
    )
'''
)
huntCase.show(10, 70)

###  Regsvr32 Non-DLL Proxy Execution Detected
##### description: Detects attempts to abuse the Microsoft control and DLL Registration Server command-line application (Regsvr32) to proxy execute a malicious DLL embedded within another filetype

In [ ]:
huntCase = spark.sql(
'''
SELECT computer_name, ParentImage, ParentCommandLine, Image, CommandLine, User, Description
FROM sparktable
WHERE (source_name = "Microsoft-Windows-Sysmon" AND event_id = 1
        AND
        (
            lower(Description) == 'microsoft(c) register server'
        )
        AND
        (
            lower(CommandLine) LIKE '%/s%'
            OR lower(CommandLine) LIKE '%-s%'
        )
        AND NOT
        (
            lower(ParentImage) LIKE '%system32%spoolsv.exe%'
        )
    )
'''
)
huntCase.show(10, 100)

###  AppVLP Proxy Execution Detected
##### description:  Detects the Proxy Execution via AppVLP

In [ ]:
huntCase = spark.sql(
'''
SELECT computer_name, ParentImage, ParentCommandLine, Image, CommandLine, User
FROM sparktable
WHERE (source_name = "Microsoft-Windows-Sysmon" AND event_id = 1
        AND
        (
            lower(ParentImage) LIKE'%ppvlp.exe%'
        )
    )
'''
)
huntCase.show(10, 70)

###  Backup File Deletion Detected
##### description:  Detects attempts to delete backup files, which may be used by adversaries to inhibit system recovery in the event of data destruction or ransomware encryption

In [ ]:
huntCase = spark.sql(
'''
SELECT computer_name, ParentImage, ParentCommandLine, Image, CommandLine, User
FROM sparktable
WHERE (source_name = "Microsoft-Windows-Sysmon" AND event_id = 1
        AND
        (
            lower(CommandLine) LIKE'%.vhd%'
            OR lower(CommandLine) LIKE'%.bac%' 
            OR lower(CommandLine) LIKE'%.bak%'
            OR lower(CommandLine) LIKE'%.wbcat%'
            OR lower(CommandLine) LIKE'%.bkf%'
            OR lower(CommandLine) LIKE'%backup%.%'
            OR lower(CommandLine) LIKE'%.set%'
            OR lower(CommandLine) LIKE'%.win%'
            OR lower(CommandLine) LIKE'%.dsk%'
        )
        AND
            lower(CommandLine) LIKE'%.del %'
        AND
        (
            lower(CommandLine) LIKE'% /s%'
            OR lower(CommandLine) LIKE'% /f%'
            OR lower(CommandLine) LIKE'% -s%'
            OR lower(CommandLine) LIKE'% -f%'
        )
    )
'''
)
huntCase.show(10, 70)

###  Potential Protocol Tunneling via Chisel
##### description:  Detects Potential Protocol Tunneling via Chisel

In [ ]:
huntCase = spark.sql(
'''
SELECT computer_name, ParentImage, ParentCommandLine, Image, CommandLine, User
FROM sparktable
WHERE (source_name = "Microsoft-Windows-Sysmon" AND event_id = 1
        AND
        (
            lower(CommandLine) LIKE'%client%'
        )
        AND
        (
            lower(CommandLine) LIKE'%socks%'
            OR lower(CommandLine) LIKE'%fingerprint%'
            OR lower(CommandLine) LIKE'%authfile%'
        )
    )
'''
)
huntCase.show(10, 70)

###  Credential Dumping via Keymgr.dll
##### description:   Detects Credential Dumping via Keymgr.dll

In [ ]:
huntCase = spark.sql(
'''
SELECT computer_name, ParentImage, ParentCommandLine, Image, CommandLine, User
FROM sparktable
WHERE (source_name = "Microsoft-Windows-Sysmon" AND event_id = 1
        AND
        (
            lower(CommandLine) LIKE'%keymgr.dll%'
            AND lower(CommandLine) LIKE'%krshowkeymgr%'
        )
    )
'''
)
huntCase.show(10, 70)

###  Msiexec HTTP Proxy Execution
##### description: Detects Msiexec HTTP Proxy Execution

In [ ]:
huntCase = spark.sql(
'''
SELECT computer_name, ParentImage, ParentCommandLine, Image, CommandLine, User
FROM sparktable
WHERE (source_name = "Microsoft-Windows-Sysmon" AND event_id = 1
        AND
        (
            lower(CommandLine) LIKE'%msiexec%'
            AND lower(CommandLine) LIKE'%http%'
        )
    )
'''
)
huntCase.show(10, 70)

###  PowerShell Execution via SQLPS.exe Detected
##### description: Detects PowerShell Execution via SQLPS.exe

In [ ]:
huntCase = spark.sql(
'''
SELECT computer_name, ParentImage, ParentCommandLine, Image, CommandLine, User
FROM sparktable
WHERE (source_name = "Microsoft-Windows-Sysmon" AND event_id = 1
        AND
        (
            lower(Description) == 'sql server powershell'
        )
    )
'''
)
huntCase.show(10, 70)

###  UAC Bypass via Fodhelper Detected
##### description: Detects UAC Bypass via Fodhelper

In [ ]:
huntCase = spark.sql(
'''
SELECT computer_name, ParentImage, ParentCommandLine, Image, CommandLine, User
FROM sparktable
WHERE (source_name = "Microsoft-Windows-Sysmon" AND event_id = 1
        AND
        (
            lower(ParentImage) LIKE '%fodhelper.exe%'
        )
    )
'''
)
huntCase.show(10, 70)

###  Proxy Execution via Verclsid Detected
##### description:  Detects Proxy Execution via Verclsid

In [ ]:
huntCase = spark.sql(
'''
SELECT computer_name, ParentImage, ParentCommandLine, Image, CommandLine, User
FROM sparktable
WHERE (source_name = "Microsoft-Windows-Sysmon" AND event_id = 1
        AND
        (
            lower(ParentImage) LIKE '%verclsid.exe%'
        )
    )
'''
)
huntCase.show(10, 70)

###  Proxy Execution via InstallUtil.exe Detected
##### description:  Detects Proxy Execution via InstallUtil.exe

In [ ]:
huntCase = spark.sql(
'''
SELECT computer_name, ParentImage, ParentCommandLine, Image, CommandLine, User
FROM sparktable
WHERE (source_name = "Microsoft-Windows-Sysmon" AND event_id = 1
        AND
        (
            lower(ParentImage) LIKE '%installutil.exe%'
        )
    )
'''
)
huntCase.show(10, 70)

###  MSDT Proxy Execution from Office Detected
##### description:  Detects MSDT Proxy Execution from Office

In [ ]:
huntCase = spark.sql(
'''
SELECT computer_name, ParentImage, ParentCommandLine, Image, CommandLine, User
FROM sparktable
WHERE (source_name = "Microsoft-Windows-Sysmon" AND event_id = 1
        AND
        (
            lower(Description) == 'diagnostics troubleshooting wizard'
        )
        AND
        (
            lower(CommandLine) LIKE'%-path%'
            AND
            (
                lower(CommandLine) LIKE'%true%'
                OR lower(CommandLine) LIKE'%/skip%'
            )
        )
    )
'''
)
huntCase.show(10, 70)

### Sdiagnhost Proxy Execution Detected
##### description:  Detects Proxy Execution via Sdiagnhost

In [ ]:
huntCase = spark.sql(
'''
SELECT computer_name, ParentImage, ParentCommandLine, Image, CommandLine, User
FROM sparktable
WHERE (source_name = "Microsoft-Windows-Sysmon" AND event_id = 1
        AND
        (
            lower(ParentImage) LIKE '%sdiagnhost.exe%'
        )
    )
'''
)
huntCase.show(10, 70)

### Suspicious RunDll32 & Control.exe invokation
##### description:  Detects suspicious invokations of RunDll32 & Control.exe .This TTP is used in ISO Embedded DLL Proxy Execution from Explorer as well.

In [ ]:
huntCase = spark.sql(
'''
SELECT computer_name, ParentImage, ParentCommandLine, Image, CommandLine, User
FROM sparktable
WHERE (source_name = "Microsoft-Windows-Sysmon" AND event_id = 1
        AND
        (
            lower(ParentImage) LIKE '%explorer.exe%'
            OR lower(ParentImage) LIKE '%cmd.exe%'
            OR lower(ParentImage) LIKE '%control.exe%'
            OR lower(ParentImage) LIKE '%powershell.exe%'
            OR lower(ParentImage) LIKE '%svchost.exe%'
        )
        AND
        (
            lower(Description) == 'windows host process (rundll32)'
            OR lower(Description) == 'windows control panel'
        )
        AND NOT
        (
            lower(ParentCommandLine) LIKE '%svchost.exe%'
            AND
            (
                lower(ParentCommandLine) LIKE '%localservice%'
                OR lower(ParentCommandLine) LIKE '%netsvcs%'
                OR lower(ParentCommandLine) LIKE '%-p%'
                OR lower(ParentCommandLine) LIKE '%-s%'
            )
        )
    )
'''
)
huntCase.show(100, 100)

### PCWRun Proxy Execution Detected
##### description:  Detects Proxy Execution via PCWRun

In [ ]:
huntCase = spark.sql(
'''
SELECT computer_name, ParentImage, ParentCommandLine, Image, CommandLine, User
FROM sparktable
WHERE (source_name = "Microsoft-Windows-Sysmon" AND event_id = 1
        AND
        (
            lower(ParentCommandLine) LIKE '%pcwutl.dll%'
            OR lower(ParentCommandLine) LIKE '%pcwutl.exe%'
            OR lower(CommandLine) LIKE '%pcwutl.dll%'
            OR lower(CommandLine) LIKE '%pcwutl.exe%'
        )
    )
'''
)
huntCase.show(10, 70)

### Safeboot Manipulation via Bcdedit/Bootcfg Detected
##### description: Safeboot Manipulation via Bcdedit/Bootcfg Detected

In [ ]:
huntCase = spark.sql(
'''
SELECT computer_name, ParentImage, ParentCommandLine, Image, CommandLine, User
FROM sparktable
WHERE (source_name = "Microsoft-Windows-Sysmon" AND event_id = 1
        AND
        (
            lower(CommandLine) LIKE '%bcdedit%'
            OR lower(CommandLine) LIKE '%bootcfg%'
        )
        AND
        (
            lower(CommandLine) LIKE '%safeboot%'
        )
    )
'''
)
huntCase.show(10, 70)